In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
# import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
import time

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

# from sympy import *
from scipy.linalg import schur, eigvals
from scipy.special import comb, perm

extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

## help functions

In [ ]:
def generate_localstatsmat_eig_desym(Nparams,Grand,rho):
    '''
    Input
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    xaxiss,yaxiss =np.arange(NE*2),np.arange(NE*2)
    Xax,Yax=np.meshgrid(xaxiss,yaxiss)
    idxwhere = np.where(Xax>Yax)
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    X,Xsym,Xasym=np.zeros((N,N)),np.zeros((N,N)),np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    XT = X.copy().T

    Xsym = np.sqrt(rho)*X.copy()
    Xsym[idxwhere]=np.sqrt(rho)*XT[idxwhere]
    
    Xasym=np.sqrt(1-rho)*np.random.randn(NE*2,NE*2)
    X=Xsym.copy()+Xasym.copy()
    # components
    Xsym[:NE,:NE],Xsym[NE:,NE:]=xee*Xsym[:NE,:NE]/np.sqrt(NE),xii*Xsym[NE:,NE:]/np.sqrt(NI)
    Xsym[:NE,NE:],Xsym[NE:,:NE]=xei*Xsym[:NE,NE:]/np.sqrt(NE),xie*Xsym[NE:,:NE]/np.sqrt(NI)
    Xasym[:NE,:NE],Xasym[NE:,NE:]=xee*Xasym[:NE,:NE]/np.sqrt(NE),xii*Xasym[NE:,NE:]/np.sqrt(NI)
    Xasym[:NE,NE:],Xasym[NE:,:NE]=xei*Xasym[:NE,NE:]/np.sqrt(NE),xie*Xasym[NE:,:NE]/np.sqrt(NI)
    X=Xsym.copy()+Xasym.copy()
    # theoretical M (grandom) matrix
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    # for i in range(2):
    #     gmat[:,i]*=at[i]
    eigvgm,eigvecgm=la.eig(gmat)
    
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii
    return (X,Xsym,Xasym,Gamp,eigvgm,eigvecgm,gaverage)

def generate_localstatsmat_eig(Nparams,Grand):
    '''
    Input 
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nt/N   
    X=np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,:NE],X[NE:,NE:]=xee*X[:NE,:NE]/np.sqrt(NE),xii*X[NE:,NE:]/np.sqrt(NI)
    X[:NE,NE:],X[NE:,:NE]=xei*X[:NE,NE:]/np.sqrt(NE),xie*X[NE:,:NE]/np.sqrt(NI)
    # theoretical M (grandom) matrix 
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    eigvgm,eigvecgm=la.eig(gmat)
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii

    return (X,Gamp,eigvgm,eigvecgm,gaverage)

def generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII):
    # mean value 
    # first use rank-1 structure
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nt/N   
    Am=np.zeros((N,N))
    Am[:NE,:NE],Am[:NE,NE:]=JEE/NE,-JEI/NI
    Am[NE:,:NE],Am[NE:,NE:]=JIE/NE,-JII/NI
    # ua,svam,vha=la.svd(Am)
    # va=vha.T
    # svam=svam[0]
    # ua,va=np.zeros((N,N)),np.zeros((N,N))
    # ua[:,0]=1.0/np.sqrt(N)
    # va[:NE,0]=JE/np.sqrt(NE)
    # va[NE:,0]=-JI/np.sqrt(NI)
    # va[:,0]/=-np.linalg.norm(va[:,0])
    # svam=np.sqrt(2*(JE**2+JI**2))
    # vha=va.T

    Jsv=np.zeros((2,2))
    Jsv[0,0],Jsv[0,1],Jsv[1,0],Jsv[1,1]=JEE,JEI,JIE,JII

    return (Am,Jsv)#,ua,svam,va)

# generate gmat, same gavg
def gmatamplitude_eig(gavgfix,typenum):
  Amplit = gavgfix*typenum
  numsample = typenum**2
  Amplitg= np.zeros(numsample)
  idxc=0
  while (1):
    if idxc>=numsample:
      Amplitg[numsample-1]=1.0-np.sum(Amplitg[:numsample-1])
      break
    p=np.random.random(1)
    Amplitg[idxc]=np.minimum(p,1.0-np.sum(Amplitg))
    if np.sum(Amplitg)>1.0:
      continue 
    elif np.sum(Amplitg)==1.0:
      break
    else:
      idxc +=1
      # Amplitg[idxc]=np.min(p,1.0-np.sum(Amplitg))
  # Amplitg=0
  Amplitg*=Amplit
  Amplitg=np.sqrt(Amplitg)
  return Amplitg

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    print(np.shape(cov))
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

## For Case 1 and 2, without symmetric connectivity
* Based on determinant lemma, calculate eigenvalue outlier (spectrum of eigenvalues)
* Based on perturbation theorem, calculate the statistics of eigenvectors (eigenvector perturbations)
* Validate the results, in the case with homogeneous random gain and the case with heterogeneous random gain 

In [ ]:
'''
Cases 1,2 without symmetric connectivity
Focus on standard deviation (variance)
Focus on second moment (self)
'''
# generate mean matrix
nrank=1
Nt=np.array([750,750])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
nrank,ntrial,neta,ngavg=1,20,10,10
## for all casesq
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)

''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

''' ---------- method 2 ------------------ '''
''' single eta '''
etaset=0*np.ones(3)
'''
## Three \bar{J} cases
'''
JI,JE,a,b=3.50,1.0,-0.6,0.6 # already recording -3.5# 2.0
a,b=0.0,0.0
# JI,JE,a,b=3.5,1.0,3.5,1.0 # already recording -3.5# 2.0
# JI,JE=0,0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
'''
Am -- J(g0=0), l0(g0=0), r0(g0=0), S=R1(B1-lambda0In-1)^(-1)L1.T
'''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(1,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])
meanReigAmE,meanReigAmI=np.mean(meigvecAm[:NE,0]),np.mean(meigvecAm[NE:,0])
meanLeigAmE,meanLeigAmI=np.mean(neigvecAm[:NE,0])/eigvAm[0],np.mean(neigvecAm[NE:,0])/eigvAm[0]
'''
for reference
'''
x0,y0=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))
'''
inverse S
'''
invB1=np.zeros((NE*2-1,NE*2-1))
for i in range(NE*2-1):
    invB1[i,i]=1/(eigvAm[i+1]-eigvAm[0])
X1,Y1=meigvecAm[:,1:].copy(),neigvecAm[:,1:].copy()
SAm = X1@invB1@Y1.T
'''
Network Setting for Iterating Following
'''
signeta=np.ones(3)
# signeta[1]*=(-1)
## heterogeneous random gain
'''
>>>>>>>>>>>>>> Case 1 and Case 2 >>>>>>>>>>>>>>>>>>>>>>>
'''
xee,xei,xie,xii=1.0,1.0,1.0,1.0 ## HOMOgeneous random gain g_kl
# xee,xei,xie,xii=1.0,0.5,0.2,0.8 ## HETEROgeneous random gain
# xee,xei,xie,xii=1.0,0.5,0.3,0.1
'''
Recording Variables
'''
# original data
eigvalseries,mvecseries,nvecseries=np.zeros((ngavg,ntrial,nrank),dtype=complex),np.zeros((ngavg,ntrial,NE*2,nrank),dtype=complex),np.zeros((ngavg,ntrial,NE*2,nrank),dtype=complex)
Reigvecseries,Leigvecseries=np.zeros((ngavg,ntrial,NE*2,N),dtype=complex),np.zeros((ngavg,ntrial,NE*2,N),dtype=complex)
Beigvseries=np.zeros((ngavg,ntrial,N),dtype=complex)
''' All recorded connectivity statistics '''
statesReig,statesLeig=np.zeros((ngavg,ntrial,3,2),dtype=complex),np.zeros((ngavg,ntrial,3,2),dtype=complex) #E,I,Total/mean,std# 
coveigvecseries=np.zeros((ngavg,ntrial,3),dtype=complex) # E, I and total
''' All random matrices for each trials '''
Xprimebasexee=np.zeros((ntrial,NE*2,NE*2)) 
# Y -- identical to X when g=1.0 ([-1])
''' Iterative Processing '''
gaverageseries=np.linspace(0.1,1.0,ngavg)
for iktrial in range(ntrial):
    '''
    for each individual trial, supposed to continuously increase g from zero to one -- see the sensibility changing
    but within one single trial, the random matrix (also random structure) keeps unchanged, thus avoid changing in random pattern
    '''
    # >>> generate the (base/ref) random matrix for each trial
    Xsym=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
    Xsym[:NE,:NE]=np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(NE)
    Xsym[NE:,NE:]=np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(NE)
    Xsym[NE:,:NE]=np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(NE)
    Xsym[:NE,NE:]=np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(NE)
    XsymT=Xsym.copy().T
    Yidendown=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
    Yidenup=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
    Xinit = Xsym.copy()
    # >>>>> Now, Xsym and XsymT keep the same while changing the random gain gaverage
    for idxeta,eta in enumerate(gaverageseries):
        #  >>> random gain continuously increases
        etaset=0.0*np.ones(3)#eta*np.ones(3)
        gaverage=eta        
        # ''' heterogeneous ETA method 1 '''
        # ## EE ##
        # asqr=(1+np.sqrt(1-etaset[0]**2))/2.0
        # aamp=np.sqrt(asqr)
        # Xinit[idxDownEE,idyDownEE]=aamp*XsymT[idxDownEE,idyDownEE].copy()+np.sqrt(1-aamp**2)*Xsym[idxDownEE,idyDownEE].copy()
        # Xinit[idxUpEE,idyUpEE]=signeta[0]*aamp*XsymT[idxUpEE,idyUpEE].copy()+signeta[0]*np.sqrt(1-aamp**2)*Xsym[idxUpEE,idyUpEE].copy()
        # ## EI IE##
        # asqr=(1+np.sqrt(1-etaset[1]**2))/2.0
        # aamp=np.sqrt(asqr)
        # Xinit[NE:,:NE]=aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        # Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+signeta[1]*np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        # ## II ##
        # asqr=(1+np.sqrt(1-etaset[2]**2))/2.0
        # aamp=np.sqrt(asqr)
        # Xinit[idxDownII,idyDownII]=aamp*XsymT[idxDownII,idyDownII].copy()+np.sqrt(1-aamp**2)*Xsym[idxDownII,idyDownII].copy()
        # Xinit[idxUpII,idyUpII]=signeta[2]*aamp*XsymT[idxUpII,idyUpII].copy()+signeta[2]*np.sqrt(1-aamp**2)*Xsym[idxUpII,idyUpII].copy()
        ''' heterogeneous ETA method 1 '''
        ## EE ##
        asqr=(1-np.sqrt(1-etaset[0]**2))/2.0 
        '''## aamp = 0, XT --> 0 X --> 1'''
        aamp=np.sqrt(asqr) 
        Xinit[:NE,:NE]=signeta[0]*aamp*XsymT[:NE,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,:NE].copy()
        ## EI IE##
        asqr=(1-np.sqrt(1-etaset[1]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,:NE]=signeta[1]*aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        ## II ##
        asqr=(1-np.sqrt(1-etaset[2]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,NE:]=signeta[2]*aamp*XsymT[NE:,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,NE:].copy()
        # ''' HETEROGENEOUS ETA method 2 '''
        # # EE#
        # Xinit[idxDownEE,idyDownEE]=Xsym[idxDownEE,idyDownEE]*np.sqrt(etaset[0])+Yidendown[idxDownEE,idyDownEE].copy()*np.sqrt(1-etaset[0])
        # Xinit[idxUpEE,idyUpEE]=signeta[0]*XsymT[idxUpEE,idyUpEE]*np.sqrt(etaset[0])+Yidenup[idxUpEE,idyUpEE].copy()*np.sqrt(1-etaset[0])
        # # EI/IE#
        # Xinit[NE:,:NE]=Xsym[NE:,:NE]*np.sqrt(etaset[1])+Yidendown[NE:,:NE].copy()*np.sqrt(1-etaset[1])
        # Xinit[:NE,NE:]=signeta[1]*XsymT[:NE,NE:]*np.sqrt(etaset[1])+Yidenup[:NE,NE:].copy()*np.sqrt(1-etaset[1])
        # # EE#
        # Xinit[idxDownII,idyDownII]=Xsym[idxDownII,idyDownII]*np.sqrt(etaset[2])+Yidendown[idxDownII,idyDownII].copy()*np.sqrt(1-etaset[2])
        # Xinit[idxUpII,idyUpII]=signeta[2]*XsymT[idxUpII,idyUpII]*np.sqrt(etaset[2])+Yidenup[idxUpII,idyUpII].copy()*np.sqrt(1-etaset[2])
        ''' THEORETICAL FULL ETA '''
        symtheofull=gaverage**2*(etaset[0]*signeta[0]+etaset[2]*signeta[2]+2*etaset[1]*signeta[1])/4.0#+(a-b)**2/(NE*2)
        # ''' heterogeneous '''
        X=Xinit.copy()
        X[:NE,:NE]*=(xee*gaverage/np.sqrt(2))
        X[:NE,NE:]*=(xei*gaverage/np.sqrt(2))
        X[NE:,:NE]*=(xie*gaverage/np.sqrt(2))
        X[NE:,NE:]*=(xii*gaverage/np.sqrt(2))

        '''
        >> start perturbed matrix
        >> * perturbed matrix: J; low rank structures: Am; random components: X
        >> * eigenvectors perturbed matrix: Reig, Leig --> renormalized by x0,y0,--> get xnorm0,ynorm0
        >> * low rank structure: x0,y0, 

        >>> statistics (recording)
        >> * mean and standard deviation of elements in eigenvectors -- E and I populations (individual)
        >> * overlap and tilting? between left and right eigenvectors --> relate to eigenvalue
        '''
        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
        symnumfull*=2*NE
        print('eta full:',symtheofull,'; num:',symnumfull)
        '''
        statistics >>>>>>>>>>> Full connectivity
        '''
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:].copy())
        # meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
        neig=neig.copy().T
        # neig = np.reshape(neig.T,(NE*2,nrank))
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
        for j in range(1,N):
            neig[:,j]*=eigvJ[j]
        ''' low rank properties m,n,eigenvalues '''
        eigvalseries[idxeta,iktrial,0],mvecseries[idxeta,iktrial,:,0],nvecseries[idxeta,iktrial,:,0]=eigvJ[0],meig[:,0].copy(),neig[:,0].copy() 
        
        Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
        ''' The first '''
        Reig=meig.copy()
        normval=np.sum(Reig*neig.copy(),axis=0)
        normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        ''' eigendecomposition properties left and right eigenvectors '''
        Leig=neig.copy()/normval.copy() 
        '''
        normalization 0 (reference)
        '''
        tildex,tildey=np.reshape(Reig[:,0].copy(),(NE*2,1)),np.reshape(Leig[:,0].copy(),(NE*2,1))
        if (np.mean(tildex[:NE])*x0[0,0])<0:
            tildex*=(-1)
            tildey*=(-1)
        xnorm0=np.sqrt(np.squeeze(tildey.T@x0)/np.squeeze(y0.T@tildex))*tildex
        ynorm0=np.sqrt(np.squeeze(tildex.T@y0)/np.squeeze(x0.T@tildey))*tildey

        Reigvecseries[idxeta,iktrial,:,:],Leigvecseries[idxeta,iktrial,:,:]=Reig.copy(),Leig.copy()
        Beigvseries[idxeta,iktrial,:]=eigvJ.copy()
        Reigvecseries[idxeta,iktrial,:,0],Leigvecseries[idxeta,iktrial,:,0]=xnorm0[:,0].copy(),ynorm0[:,0].copy()

        '''  RECORDING STATES 
        * statesReig/Leig[grandom,trial,E/I/Total,mean/std]

        '''
        statesReig[idxeta,iktrial,0,0],statesReig[idxeta,iktrial,1,0]=np.mean(Reigvecseries[idxeta,iktrial,:NE,0]),np.mean(Reigvecseries[idxeta,iktrial,NE:,0]). 
        statesReig[idxeta,iktrial,2,0]=np.mean(Reigvecseries[idxeta,iktrial,:,0])
        statesReig[idxeta,iktrial,0,1],statesReig[idxeta,iktrial,1,1]=np.std(Reigvecseries[idxeta,iktrial,:NE,0]),np.std(Reigvecseries[idxeta,iktrial,NE:,0])
        statesReig[idxeta,iktrial,2,1]=np.std(Reigvecseries[idxeta,iktrial,:,0])
       
        statesLeig[idxeta,iktrial,0,0],statesLeig[idxeta,iktrial,1,0]=np.mean(Leigvecseries[idxeta,iktrial,:NE,0]),np.mean(Leigvecseries[idxeta,iktrial,NE:,0])
        statesLeig[idxeta,iktrial,2,0]=np.mean(Leigvecseries[idxeta,iktrial,:,0])
        statesLeig[idxeta,iktrial,0,1],statesLeig[idxeta,iktrial,1,1]=np.std(Leigvecseries[idxeta,iktrial,:NE,0]),np.std(Leigvecseries[idxeta,iktrial,NE:,0])
        statesLeig[idxeta,iktrial,2,1]=np.std(Leigvecseries[idxeta,iktrial,:,0])

        ''' covariance '''
        '''
        Note, we should change the normalization for the first item to the (Refer) normalization 
        tildeReig,tildeLeig
        normalization 0
        '''
        ## normalization 0 reference
        tildeReig,tildeLeig=np.squeeze(Reigvecseries[idxeta,iktrial,:,:]),np.squeeze(Leigvecseries[idxeta,iktrial,:,:])
        reig,leig=np.reshape(tildeReig[:,0].copy(),(NE*2,1)),np.reshape(tildeLeig[:,0].copy(),(NE*2,1))
        ## mean at eact point
        rEceneig,rIceneig = np.mean(reig[:NE,0]),np.mean(reig[NE:,0])
        lEceneig,lIceneig = np.mean(leig[:NE,0]),np.mean(leig[NE:,0])
        ## random components
        rrandeig,lrandeig=reig.copy(),leig.copy()
        rrandeig[:NE,0],rrandeig[NE:,0]=rrandeig[:NE,0]-rEceneig,rrandeig[NE:,0]-rIceneig
        lrandeig[:NE,0],lrandeig[NE:,0]=lrandeig[:NE,0]-lEceneig,lrandeig[NE:,0]-lIceneig

        coveigvecseries[idxeta,iktrial,0]=np.reshape(rrandeig[:NE,0],(NE,1)).T@np.reshape(lrandeig[:NE,0],(NE,1))
        coveigvecseries[idxeta,iktrial,1]=np.reshape(rrandeig[NE:,0],(NE,1)).T@np.reshape(lrandeig[NE:,0],(NE,1))
        coveigvecseries[idxeta,iktrial,2]=coveigvecseries[idxeta,iktrial,0]+coveigvecseries[idxeta,iktrial,1]
        ''' record the random derivative '''
        if idxeta == ngavg-1:
            print('recording!!!')
            Xprimebasexee[iktrial,:,:]=X.copy()


* Recording the PARAMETERS, VARIABLES, DATA

In [ ]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE
Randgain=np.zeros(4)
Randgain[0],Randgain[1],Randgain[2],Randgain[3]=xee,xei,xie,xii

np.savez(now+'_perturbationHomoZeroeta_data',x0=x0,y0=y0,Jparameters=Jparameters,Randgain=Randgain,gaverageseries=gaverageseries,etaset=etaset,signeta=signeta,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries[:,:,:,0],Leigvecseries=Leigvecseries[:,:,:,0],Beigvseries=Beigvseries[:,:,0],meigvecAm=meigvecAm[:,0],neigvecAm=neigvecAm[:,0],eigvAm=eigvAm,statesReig=statesReig,statesLeig=statesLeig,coveigvecseries=coveigvecseries,coveigvecAmseries=coveigvecAmseries,Xprimebasexee=Xprimebasexee)

* EIGENSPECTRA (eigenbulk and eigenvalue outlier)

In [ ]:
'''
without symmetry, 
eigvAm
'''
idxgavg,idxtrial=9,8
figtspt,axtspt=plt.subplots(figsize=(6,3))
axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,1:]),np.imag(Beigvseries[idxgavg,idxtrial,1:]),s=5,c='blue',alpha=0.125)  # >>>>>>>>>>>>>>>>>>>.
axtspt.scatter(np.real(Beigvseries[idxgavg,:,0]),np.imag(Beigvseries[idxgavg,:,0])-0.0,s=20,c='orange',marker='^',alpha=0.5)  # >>>>>>>>>>>>>>>>>>>
axtspt.set_aspect('equal')
axtspt.plot(np.ones(2)*eigvAm[0],np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
axtspt.spines['right'].set_color('none')
axtspt.spines['top'].set_color('none')
#location of X 
axtspt.xaxis.set_ticks_position('bottom')
axtspt.spines['bottom'].set_position(('data', 0))

aee,aei,aie,aii=xee,xei,xie,xii
print(xee,xei,xie,xii)
idxtrial=10
eta=0
xticks = np.linspace(-1.3,1.3,2)
xlims = [-1.3,1.3]
yticks = np.linspace(-1.3,1.3,2)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
# first do not multiply at
ahomo=gaverageseries[idxgavg]
xee_,xei_,xie_,xii_=ahomo*aee/np.sqrt(2),ahomo*aei/np.sqrt(2),ahomo*aie/np.sqrt(2),ahomo*aii/np.sqrt(2)
## effective gain only use ONE alpha(portion), average gain use alphas of sender and receiver
gmat = np.array([[xee_**2,xei_**2],[xie_**2,xii_**2]])
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
r_g = np.sqrt(r_g2)
print(r_g)
## >> zero symmetry >>
eta=0
longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
xr = longaxis*np.cos(theta)
yr = shortaxis*np.sin(theta)
axtspt.plot(xr, yr, color="darkorange", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>>>>>>>>>
avgg=np.sum(np.sum(gmat))*2/4.0
avgg=np.sqrt(avgg)
xa = avgg*np.cos(theta)
ya = avgg*np.sin(theta)
axtspt.plot(xa, ya, color="gray", linewidth=2.5,linestyle='--',label=r'ellipse')
print(avgg,gaverage_)



## individual perturbations of left and right eigenvectors (predictive accuracy)
* calculate the predicted perturbations (theo)
* comparing with numerical perturbations (num)

In [ ]:
'''
>>>>>>>>>>>>>>>>> Compare the individual response (not statistics), NOTE:
To make direct comparison between numerical and perturbation theorem, keep the same random matrix for each trial
choose one trial -- determine the random derivative -- calculate the individual elements in vector

'''

'''
# loading data
# strfile='2021-03-13-15_20_04_perturbationHomoZeroeta_data'# homogeneous random gain
strfile='2021-03-13-15_10_37_perturbationHeterZeroeta_data' # heterogeneous random gain
data = np.load(strfile+'.npz')
gaverageseries = data['gaverageseries']
x0,y0=data['x0'],data['y0']
meigvecAm,neigvecAm=data['meigvecAm'],data['neigvecAm']
# perturbed
Leigvecseries,Reigvecseries=data['Leigvecseries'],data['Reigvecseries']
Xprimebasexee=data['Xprimebasexee']
eigvAm=data['eigvAm']
ngavg,ntrial,N=np.shape(Leigvecseries)
Leigvecseries=np.expand_dims(Leigvecseries,axis=3)
Reigvecseries=np.expand_dims(Reigvecseries,axis=3)
NE=int(N/2.0)
'''

'''
calculate S0
'''
xticks = np.linspace(-0.08,0.08,2)
xlims = [-0.08,0.08]
yticks = np.linspace(-0.08,0.08,2)
ylims = [-0.08,0.08]
Z = np.zeros((NE*2-1,NE*2-1))
for i in range(NE*2-1):
    b1=eigvAm[i+1]
    Z[i,i]=1/(-eigvAm[0])
S0=np.zeros((NE*2,NE*2))
R1,L1=np.squeeze(meigvecAm[:,1:]),np.squeeze(neigvecAm[:,1:])
S0=R1@Z@L1.T

'''
# CHOOSE ONE TRIAL
'''
## EIGENVECTOR USED HERE :0 -- the last dimension, not reloading from .npy
idxtrial,idxtrial_=8,3
fig,ax=plt.subplots(2,3,figsize=(6,4))
Xprime = np.squeeze(Xprimebasexee[idxtrial,:,:])
idxgsamples=np.array([1,5,9])
gsamples=gaverageseries[idxgsamples]
rperturb_num,lperturb_num = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rperturb_the,lperturb_the = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rprime_the,lprime_the = np.squeeze(Xprime@x0/eigvAm[0]),np.squeeze(Xprime.T@y0/eigvAm[0])
for i in range(len(idxgsamples)):
    lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial,:,0])-np.squeeze(y0)
    rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial,:,0])-np.squeeze(x0)

    lperturb_the[:,i] = lprime_the*gsamples[i]
    rperturb_the[:,i] = rprime_the*gsamples[i]
    ax[0,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[1,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[0,i].scatter(np.real(rperturb_the[:,i]),rperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[1,i].scatter(np.real(lperturb_the[:,i]),lperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[0,i].set_xticks([])
    ax[0,i].set_yticks([])
    ax[1,i].set_xticks([])
    ax[1,i].set_yticks([])

for i in range(3):
    ax[0,i].set_xlim(xlims)
    ax[0,i].set_ylim(ylims)
    ax[1,i].set_xlim(xlims)
    ax[1,i].set_ylim(ylims)
ax[0,0].set_yticks(yticks)
ax[1,0].set_yticks(yticks)
for i in range(3):
    ax[1,i].set_xticks(xticks)
# print(np.shape(rprime_the),np.shape(rprime_num))
# ax.scatter(np.real(rprime_the),rprime_num,c='gray',alpha=0.5)
# ax.set_xlim([-0.05,0.05])
# ax.set_ylim([-0.05,0.05])
'''
# CHOOSE TWO TRIALS
'''
idxtrial,idxtrial_=8,3
fig,ax=plt.subplots(2,3,figsize=(6,4))
Xprime = np.squeeze(Xprimebasexee[idxtrial,:,:])
idxgsamples=np.array([1,5,9])
gsamples=gaverageseries[idxgsamples]
rperturb_num,lperturb_num = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rperturb_the,lperturb_the = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rprime_the,lprime_the = np.squeeze(Xprime@x0/eigvAm[0]),np.squeeze(Xprime.T@y0/eigvAm[0])
for i in range(len(idxgsamples)):
    lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial_,:,0])-np.squeeze(y0)
    rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial_,:,0])-np.squeeze(x0)

    lperturb_the[:,i] = lprime_the*gsamples[i]
    rperturb_the[:,i] = rprime_the*gsamples[i]
    ax[0,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[1,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[0,i].scatter(np.real(rperturb_the[:,i]),rperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[1,i].scatter(np.real(lperturb_the[:,i]),lperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[0,i].set_xticks([])
    ax[0,i].set_yticks([])
    ax[1,i].set_xticks([])
    ax[1,i].set_yticks([])



for i in range(3):
    ax[0,i].set_xlim(xlims)
    ax[0,i].set_ylim(ylims)
    ax[1,i].set_xlim(xlims)
    ax[1,i].set_ylim(ylims)
ax[0,0].set_yticks(yticks)
ax[1,0].set_yticks(yticks)
for i in range(3):
    ax[1,i].set_xticks(xticks)
# print(np.shape(rprime_the),np.shape(rprime_num))
# ax.scatter(np.real(rprime_the),rprime_num,c='gray',alpha=0.5)
# ax.set_xlim([-0.05,0.05])
# ax.set_ylim([-0.05,0.05])

## population statistics 
* first determint lemma for eigenvalue outlier -- no change
* perturbation theorem for variance of eigenvalue
* perturbation theorem for variance of left and right eigenvectors

In [ ]:
'''
### >>>>>>>>>>>>>>>>> Statistics Linearly Change >>>>>>>>>>>>>>>>
Case 1,2 focus on second-order statstics WITHIN (left/right) eigenvectors
Calculate the statistics of eigenvalue, eigenvectors
  * mean, variance of the perturbation of eigenvalue \mu(derivative of lambda),\sigma(derivative of lambda)
'''
'''
Variables:
    std_perturb_Lambda_num (1)
    std_perturb_Lambda_theo (1)
    std_perturbLambda_linear: reconstruct np.zeros(ngavg)
    std_perturbLambda_linear_theo=std_perturb_Lambda_theo*gaverageseries
'''
## calculating the statistics of perturbation lambda
# Coding uses only TWO POINT(starting S and ending E) [linear assumption]
std_perturb_Lambda_numSE  = np.std(eigvalseries[ngavg-1,:,0])
std_perturb_Lambda_theo = np.sqrt((JE**2*(xee**2+xei**2)+JI**2*(xie**2+xii**2))/(2*NE)/(JE-JI)**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbLambda_linear_theo=std_perturb_Lambda_theo*gaverageseries
std_perturbLambda_linear_numSE=std_perturb_Lambda_numSE*gaverageseries
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Lambda_num = np.std(Beigvseries[:,:,0],axis=1)
figE,axE=plt.subplots(figsize=(4,4))
cm='rbg'
axE.plot(gaverageseries,std_perturb_Lambda_num,color=cm[1],linewidth=1.5,label='numerical')
# polyfitting
p=np.polyfit(gaverageseries,std_perturb_Lambda_num,1)
std_perturbLambda_linear_num = np.zeros(ngavg)
std_perturbLambda_linear_num = p[0]*gaverageseries+p[1]
# axE.plot(gaverageseries,std_perturbLambda_linear,color=cm[2],linestyle='--',linewidth=1.5,alpha=0.5,label='1-order polynomial')
axE.plot(gaverageseries,std_perturbLambda_linear_theo,color='darkgreen',linewidth=1.5,label='analytical')
print('std delta_lambda numerical:',p[0])
axE.set_title(r'$\sigma_{\lambda}$')

'''
plt and ax
'''
ylims=[-0.00,0.08]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,0.08,2)
xticks = np.linspace(0.0,1.0,2)
axE.set_xlim(xlims)
axE.set_xticks(xticks)
axE.set_ylim(ylims)
axE.set_yticks(yticks)
# ax.legend()

'''
right eigenvector
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
std_perturb_Reig_numSE,std_perturb_Reig_theo=np.zeros(2),np.zeros(2)
std_perturb_Reig_numSE[0],std_perturb_Reig_numSE[1] = np.mean(statesReig[ngavg-1,:,0,1]),np.mean(statesReig[ngavg-1,:,1,1])
std_perturb_Reig_theo[0],std_perturb_Reig_theo[1] = np.sqrt(((xee)**2+(xei)**2)/(2*2*NE)/eigvAm[0]**2),np.sqrt(((xie)**2+(xii)**2)/(2*2*NE)/eigvAm[0]**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbReig_linear_theo,std_perturbReig_linear_numSE=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturbReig_linear_theo[:,0]=std_perturb_Reig_theo[0]*gaverageseries
std_perturbReig_linear_theo[:,1]=std_perturb_Reig_theo[1]*gaverageseries
std_perturbReig_linear_numSE[:,0]=std_perturb_Reig_numSE[0]*gaverageseries
std_perturbReig_linear_numSE[:,1]=std_perturb_Reig_numSE[1]*gaverageseries

print('std of perturbation right eigenvector, num:',std_perturb_Reig_numSE,'; theo:',std_perturb_Reig_theo)
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Reig_num,std_perturbReig_linear_num=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturb_Reig_num[:,0],std_perturb_Reig_num[:,1] = np.mean(statesReig[:,:,0,1],axis=1),np.mean(statesReig[:,:,1,1],axis=1)

figR0,axR0=plt.subplots(2,1,figsize=(4,4))
cm='rbg'
# >>>>>>>> numerical solution
axR0[0].scatter(gaverageseries,std_perturb_Reig_num[:,0],color='darkred',label='numerical')
axR0[1].scatter(gaverageseries,std_perturb_Reig_num[:,1],color='darkblue',label='numerical')
axR0[0].scatter(gaverageseries,std_perturbReig_linear_numSE[:,0],color='orange',label='numerical')
axR0[1].scatter(gaverageseries,std_perturbReig_linear_numSE[:,1],color='cyan',label='numerical')
# >>>>>>> Theoretical solutions
axR0[0].plot(gaverageseries,std_perturbReig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical')
axR0[1].plot(gaverageseries,std_perturbReig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical')
axR0[0].set_title(r'$\sigma_{r}^E$')
axR0[1].set_title(r'$\sigma_{r}^I$')

figR,axR=plt.subplots(2,1,figsize=(4,4))
cm='rbg'
# >>>>>>>> numerical solution
for i in range(ntrial):
    axR[0].scatter(gaverageseries,statesReig[:,i,0,1],s=10,c='darkred',marker='^',label='numerical')
    axR[1].scatter(gaverageseries,statesReig[:,i,1,1],s=10,c='darkblue',marker='^',label='numerical')
# >>>>>>> Theoretical solutions
axR[0].plot(gaverageseries,std_perturbReig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical')
axR[1].plot(gaverageseries,std_perturbReig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical')
axR[0].set_title(r'$\sigma_{r}^E$')
axR[1].set_title(r'$\sigma_{r}^I$')

ylims=[-0.00,0.015]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,0.015,2)
xticks = np.linspace(0.0,1.0,2)
for i in range(2):
    axR[i].set_xlim(xlims)
    axR[i].set_xticks([])
    axR[i].set_ylim(ylims)
    axR[i].set_yticks(yticks)

    axR0[i].set_xlim(xlims)
    axR0[i].set_xticks([])
    axR0[i].set_ylim(ylims)
    axR0[i].set_yticks(yticks)
axR[1].set_xticks(xticks)
axR0[1].set_xticks(xticks)

'''
left eigenvector
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
std_perturb_Leig_numSE,std_perturb_Leig_theo=np.zeros(2),np.zeros(2)
std_perturb_Leig_numSE[0],std_perturb_Leig_numSE[1] = np.mean(statesLeig[ngavg-1,:,0,1]),np.mean(statesLeig[ngavg-1,:,1,1])
std_perturb_Leig_theo[0],std_perturb_Leig_theo[1] = np.sqrt(2*((JE*xee)**2+(JI*xie)**2)/(2*NE)/(JE-JI)**2/eigvAm[0]**2),np.sqrt(2*((JE*xei)**2+(JI*xii)**2)/(2*NE)/(JE-JI)**2/eigvAm[0]**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbLeig_linear_theo,std_perturbLeig_linear_numSE=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturbLeig_linear_theo[:,0]=std_perturb_Leig_theo[0]*gaverageseries
std_perturbLeig_linear_theo[:,1]=std_perturb_Leig_theo[1]*gaverageseries
std_perturbLeig_linear_numSE[:,0]=std_perturb_Leig_numSE[0]*gaverageseries
std_perturbLeig_linear_numSE[:,1]=std_perturb_Leig_numSE[1]*gaverageseries

print('std of perturbation left eigenvector, num:',std_perturb_Leig_numSE,'; theo:',std_perturb_Leig_theo)
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Leig_num,std_perturbLeig_linear_num=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturb_Leig_num[:,0],std_perturb_Leig_num[:,1] = np.mean(statesLeig[:,:,0,1],axis=1),np.mean(statesLeig[:,:,1,1],axis=1)

figL0,axL0=plt.subplots(2,1,figsize=(4,4))
cm='rbg'
# >>>>>>>> numerical solution
axL0[0].scatter(gaverageseries,std_perturb_Leig_num[:,0],color='darkred',label='numerical')
axL0[1].scatter(gaverageseries,std_perturb_Leig_num[:,1],color='darkblue',label='numerical')
axL0[0].scatter(gaverageseries,std_perturbLeig_linear_numSE[:,0],color='orange',label='numerical')
axL0[1].scatter(gaverageseries,std_perturbLeig_linear_numSE[:,1],color='cyan',label='numerical')
# >>>>>>> Theoretical solutions
axL0[0].plot(gaverageseries,std_perturbLeig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical')
axL0[1].plot(gaverageseries,std_perturbLeig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical')
axL0[0].set_title(r'$\sigma_{l}^E$')
axL0[1].set_title(r'$\sigma_{l}^I$')

figL,axL=plt.subplots(2,1,figsize=(4,4))
cm='mg'#'rb'#
# >>>>>>>> numerical solution
for i in range(ntrial):
    axL[0].scatter(gaverageseries,statesLeig[:,i,0,1],s=10,c='darkred',marker='^',label='numerical')
    axL[1].scatter(gaverageseries,statesLeig[:,i,1,1],s=10,c='darkblue',marker='^',label='numerical')
# >>>>>>> Theoretical solutions
axL[0].plot(gaverageseries,std_perturbLeig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical')
axL[1].plot(gaverageseries,std_perturbLeig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical')
axL[0].set_title(r'$\sigma_{l}^E$')
axL[1].set_title(r'$\sigma_{l}^I$')

ylims=[-0.00,0.025]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,0.025,2)
xticks = np.linspace(0.0,1.0,2)
for i in range(2):
    axL[i].set_xlim(xlims)
    axL[i].set_xticks([])
    axL[i].set_ylim(ylims)
    axL[i].set_yticks(yticks)

    axL0[i].set_xlim(xlims)
    axL0[i].set_xticks([])
    axL0[i].set_ylim(ylims)
    axL0[i].set_yticks(yticks)
axL[1].set_xticks(xticks)
axL0[1].set_xticks(xticks)

## For Case 3 and 4, with symmetric connectivity(FULL, HOMOGENEOUS SYMMETRY)
* Based on determinant lemma, calculate eigenvalue outlier (spectrum of eigenvalues)
* Based on perturbation theorem, calculate the statistics of eigenvectors (eigenvector perturbations, MEAN)
* Validate the results, in the case with homogeneous random gain and the case with heterogeneous random gain 

In [ ]:
'''
WITH SYMMETRY
'''
'''
Cases 3,4 with symmetry
Focus on mean 
Focus on second moment
Focus on derivative on 
'''
# generate mean matrix
nrank=1
Nt=np.array([750,750])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
'''
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
'''
nrank,ntrial,neta,ngavg=1,20,10,10

## for all casesq
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)

''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

''' 
---------- method 2 ------------------ 
'''
''' ## Three \bar{J} cases '''
JI,JE,a,b=3.5,1.0,0,0#1.0,3.5,0,0#3.5,1.00,-0.6,0.6 # already recording -3.5# 2.0
a,b=0.0,0.0
# JI,JE,a,b=3.5,1.0,3.5,1.0 # already recording -3.5# 2.0
# JI,JE=0,0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(1,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe
''' for reference '''
xAm,yAm=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))

'''
##>>>>>>>>>>> g0!=0, one example >>>>>>>>
'''
gaverage=1.0
# X0=np.random.randn(NE*2,NE*2)*gaverage/np.sqrt(2*NE) # /np.sqrt(2) no later
X0=np.zeros((NE*2,NE*2))
X0[:NE,:NE]=np.reshape(np.random.randn(NE*NE),(NE,NE))*gaverage/np.sqrt(2*NE)
X0[NE:,NE:]=np.reshape(np.random.randn(NE*NE),(NE,NE))*gaverage/np.sqrt(2*NE)
X0[NE:,:NE]=np.reshape(np.random.randn(NE*NE),(NE,NE))*gaverage/np.sqrt(2*NE)
X0[:NE,NE:]=np.reshape(np.random.randn(NE*NE),(NE,NE))*gaverage/np.sqrt(2*NE)
Xsym=X0.copy()
XsymT=Xsym.copy().T
J0=Am.copy()+X0.copy()
## >>>>>> calculate the original \bar{m} and \bar{n} 
eigvJ0,eigrvecJ0=la.eig(J0)
inveigrvecJ0=la.inv(eigrvecJ0)
meig0 = np.squeeze(eigrvecJ0[:,:].copy())
neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
neig0=neig0.copy().T
## test neig0*meig0=1
print('test normal:',np.sum(neig0[:,0]*meig0[:,0]))
for i in range(nrank):
    neig0[:,i]*=eigvJ0[i]
for j in range(1,N):
    neig0[:,j]*=eigvJ0[j]
## >>>>> eigendecomposition 
leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
''' The first '''
reig0=meig0.copy()
normval=np.sum(reig0*neig0.copy(),axis=0)
normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
leig0=neig0.copy()/normval.copy()
''' calculate s0'''
Z = np.zeros((NE*2-1,NE*2-1))
for i in range(NE*2-1):
    b1=eigvJ0[i+1]
    Z[i,i]=1/(b1-eigvJ0[0])
S0=reig0[:,1:]@Z@leig0[:,1:].T
print('test normal 2:',np.sum(leig0[:,0]*reig0[:,0]))
print('test normal 3:',np.sum(meig0[:,0]*neig0[:,0]))
print('eigenvalues:',eigvJ0[:nrank])
'''## for reference '''
x0,y0=np.reshape(reig0[:,0],(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
''' ## normalization 0 (reference) absolute reference is xAm, yAm '''
tildex,tildey=x0[:,0].copy(),y0[:,0].copy()
## make sure the E in x is positive, if negative then all x and y should change sign
if (np.mean(tildex[:NE])*xAm[0,0])<0:
    tildex*=(-1)
    tildey*=(-1)
x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
## >>>> renormalization, mu_l and mu_r are unchanged
x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey
x0,y0=np.reshape(x0,(NE*2,1)),np.reshape(y0,(NE*2,1))
'''
##>>>>>>>>>>> end >>>>>>>>
'''

'''
Network Setting for Iterating Following
'''
signeta=np.ones(3)
# signeta[1]*=(-1)
## heterogeneous random gain
''' >>>>>>>>>>>>>> Case 1 and Case 2 >>>>>>>>>>>>>>>>>>>>>>> '''
xee,xei,xie,xii=1.0,1.0,1.0,1.0 ## HOMOgeneous random gain g_kl
# xee,xei,xie,xii=1.0,0.5,0.2,0.8 ## HETEROgeneous random gain
coeffeta=np.array([1.0,0.2,0.6]) ## HETERogeneous symmetry D_{\rho}
# coeffeta=np.array([1.0,0.8,1.0])
# coeffeta=np.array([1.0,1.0,1.0])

'''
Recording Variables
'''
# original data
eigvalseries=np.zeros((ngavg,ntrial,N),dtype=complex)
Reigvecseries,Leigvecseries=np.zeros((ngavg,ntrial,NE*2),dtype=complex),np.zeros((ngavg,ntrial,NE*2),dtype=complex)
Beigvseries=np.zeros((ngavg,ntrial,N),dtype=complex)
''' All recorded connectivity statistics '''
statesReig,statesLeig=np.zeros((ngavg,ntrial,3,2),dtype=complex),np.zeros((ngavg,ntrial,3,2),dtype=complex) #E,I,Total/mean,std# 
coveigvecseries=np.zeros((ngavg,ntrial,3),dtype=complex) # E, I and total
''' All random matrices for each trials '''
## Not X total (symmetry), nut Y independent
gYbasexee=np.zeros((ntrial,NE*2,NE*2),dtype=complex)

''' Iterative Processing '''
etaseries=np.linspace(0.0,1.0,neta)
for iktrial in range(ntrial):
    '''
    ##>>>>>>>>>>> g0!=0, >>>>>>>>
    '''
    Xsym=np.random.randn(NE*2,NE*2)*gaverage/np.sqrt(2*NE) # /np.sqrt(2) no later
    Xsym[:NE,:NE]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[:NE,NE:]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[NE:,:NE]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[NE:,NE:]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    XsymT=Xsym.copy().T

    J0=Am.copy()+X0.copy()
    ''' # calculate the original \bar{m} and \bar{n} '''
    eigvJ0,eigrvecJ0=la.eig(J0)
    inveigrvecJ0=la.inv(eigrvecJ0)
    meig0 = np.squeeze(eigrvecJ0[:,:].copy())
    neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
    neig0=neig0.copy().T
    print('test normal:',np.sum(neig0[:,0]*meig0[:,0]))
    for i in range(nrank):
        neig0[:,i]*=eigvJ0[i]
    for j in range(1,N):
        neig0[:,j]*=eigvJ0[j]

    leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
    ''' The first '''
    reig0=meig0.copy()
    normval=np.sum(reig0*neig0.copy(),axis=0)
    normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
    leig0=neig0.copy()/normval.copy()
    ''' calculate s0'''
    Z = np.zeros((NE*2-1,NE*2-1))
    for i in range(NE*2-1):
        b1=eigvJ0[i+1]
        Z[i,i]=1/(b1-eigvJ0[0])
    S0=reig0[:,1:]@Z@leig0[:,1:].T
    print('test normal 2:',np.sum(leig0[:,0]*reig0[:,0]))
    print('test normal 3:',np.sum(meig0[:,0]*neig0[:,0]))
    print('eigenvalues:',eigvJ0[:nrank])
    '''## for reference '''
    x0,y0=np.reshape(reig0[:,0],(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
    ''' ## normalization 0 (reference) absolute reference is xAm, yAm '''
    tildex,tildey=np.reshape(x0[:,0].copy(),(NE*2,1)),np.reshape(y0[:,0].copy(),(NE*2,1))
    ## make sure the E in x is positive, if negative then all x and y should change sign
    if (np.mean(tildex[:NE])*xAm[0,0])<0:
        tildex*=(-1)
        tildey*=(-1)
    x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
    x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
    y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey
    x0,y0=np.reshape(x0,(NE*2,1)),np.reshape(y0,(NE*2,1))
    for idxeta,eta in enumerate(etaseries):
        etaset=eta*np.ones(3)#np.zeros(3)#eta*np.ones(3)
        for icoeff in range(3):
          etaset[icoeff]*=coeffeta[icoeff]
        # etaset[0]*=0.1
        Xinit = Xsym.copy()

        ''' heterogeneous ETA method 1 '''
        ## EE ##
        asqr=(1-np.sqrt(1-etaset[0]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[:NE,:NE]=signeta[0]*aamp*XsymT[:NE,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,:NE].copy()
        ## EI IE##
        asqr=(1-np.sqrt(1-etaset[1]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,:NE]=signeta[1]*aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        ## II ##
        asqr=(1-np.sqrt(1-etaset[2]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,NE:]=signeta[2]*aamp*XsymT[NE:,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,NE:].copy()
        # ### >>>>>>>>. half Exc sym >>>>>>>>>.
        # hNE=int(5*NE/3)
        # asqr=(1-np.sqrt(1-eta**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        # aamp=np.sqrt(asqr)
        # Xinit[:hNE,:hNE]=signeta[0]*aamp*XsymT[:hNE,:hNE].copy()+np.sqrt(1-aamp**2)*Xsym[:hNE,:hNE].copy()
        ''' THEORETICAL FULL ETA '''
        symtheofull=gaverage**2*(etaset[0]*signeta[0]+etaset[2]*signeta[2]+2*etaset[1]*signeta[1])/4.0#+(a-b)**2/(NE*2)
        X=Xinit.copy()
        # incase heterogeneous
        X[:NE,:NE]*=(xee)
        X[:NE,NE:]*=(xei)
        X[NE:,:NE]*=(xie)
        X[NE:,NE:]*=(xii)

        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
        symnumfull*=2*NE
        print('compare degree of symmetry:',symnumfull,'--num; ',etaset*signeta,'--setting')
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:].copy())
        # meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
        neig=neig.copy().T
        print('test normal 4:',np.sum(neig[:,0]*meig[:,0]))
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
        for j in range(1,N):
            neig[:,j]*=eigvJ[j]
        ## >>> for low rank framework m and n
        eigvalseries[idxeta,iktrial,:]=eigvJ[:]
        
        ## >>> for eigendecomposition 
        Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
        ''' The first '''
        Reig=meig.copy()
        normval=np.sum(Reig*neig.copy(),axis=0)
        normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        Leig=neig.copy()/normval.copy()
        print('test normal 5:',np.sum(Leig[:,0]*Reig[:,0]))
        '''
        normalization 0 (reference)
        '''
        tildex,tildey=Reig[:,0].copy(),Leig[:,0].copy()
        if (np.mean(tildex[:NE])*xAm[0,0])<0:
            tildex*=(-1)
            tildey*=(-1)
        xnorm0=np.sqrt(np.squeeze(tildey.T@x0)/np.squeeze(y0.T@tildex))*tildex
        ynorm0=np.sqrt(np.squeeze(tildex.T@y0)/np.squeeze(x0.T@tildey))*tildey

        Beigvseries[idxeta,iktrial,:]=eigvJ.copy()
        Reigvecseries[idxeta,iktrial,:],Leigvecseries[idxeta,iktrial,:]=xnorm0[:].copy(),ynorm0[:].copy()

        '''  RECORDING STATES '''
        statesReig[idxeta,iktrial,0,0],statesReig[idxeta,iktrial,1,0]=np.mean(Reigvecseries[idxeta,iktrial,:NE]),np.mean(Reigvecseries[idxeta,iktrial,NE:])
        statesReig[idxeta,iktrial,2,0]=np.mean(Reigvecseries[idxeta,iktrial,:])
        statesReig[idxeta,iktrial,0,1],statesReig[idxeta,iktrial,1,1]=np.std(Reigvecseries[idxeta,iktrial,:NE]),np.std(Reigvecseries[idxeta,iktrial,NE:])
        statesReig[idxeta,iktrial,2,1]=np.std(Reigvecseries[idxeta,iktrial,:])
       
        statesLeig[idxeta,iktrial,0,0],statesLeig[idxeta,iktrial,1,0]=np.mean(Leigvecseries[idxeta,iktrial,:NE]),np.mean(Leigvecseries[idxeta,iktrial,NE:])
        statesLeig[idxeta,iktrial,2,0]=np.mean(Leigvecseries[idxeta,iktrial,:])
        statesLeig[idxeta,iktrial,0,1],statesLeig[idxeta,iktrial,1,1]=np.std(Leigvecseries[idxeta,iktrial,:NE]),np.std(Leigvecseries[idxeta,iktrial,NE:])
        statesLeig[idxeta,iktrial,2,1]=np.std(Leigvecseries[idxeta,iktrial,:])

        ''' covariance '''
        '''
        Note, we should change the normalization for the first item to the (Refer) normalization 
        tildeReig,tildeLeig
        normalization 0
        '''
        ## normalization 0 reference
        tildeReig,tildeLeig=np.squeeze(Reigvecseries[idxeta,iktrial,:]),np.squeeze(Leigvecseries[idxeta,iktrial,:])
        reig,leig=np.reshape(tildeReig[:].copy(),(NE*2,1)),np.reshape(tildeLeig[:].copy(),(NE*2,1))
        ## mean at eact point
        rEceneig,rIceneig = np.mean(reig[:NE]),np.mean(reig[NE:])
        lEceneig,lIceneig = np.mean(leig[:NE]),np.mean(leig[NE:])
        ## random components
        rrandeig,lrandeig=reig.copy(),leig.copy()
        rrandeig[:NE],rrandeig[NE:]=rrandeig[:NE]-rEceneig,rrandeig[NE:]-rIceneig
        lrandeig[:NE],lrandeig[NE:]=lrandeig[:NE]-lEceneig,lrandeig[NE:]-lIceneig
        coveigvecseries[idxeta,iktrial,0]=np.reshape(rrandeig[:NE],(NE,1)).T@np.reshape(lrandeig[:NE],(NE,1))
        coveigvecseries[idxeta,iktrial,1]=np.reshape(rrandeig[NE:],(NE,1)).T@np.reshape(lrandeig[NE:],(NE,1))
        coveigvecseries[idxeta,iktrial,2]=coveigvecseries[idxeta,iktrial,0]+coveigvecseries[idxeta,iktrial,1]
        ''' record the random derivative '''
        if idxeta == 0:
            print('recording!!!')
            ## log: there is a bug, X is the total random xconn, including symmetry
            gYbasexee[iktrial,:,:]=Xsym.copy()


* recording PARAMETERS, VARIABLES and DATA

In [ ]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE
Randgain=np.zeros(4)
Randgain[0],Randgain[1],Randgain[2],Randgain[3]=xee,xei,xie,xii

np.savez(now+'_perturbationHetereta_data',x0=x0,y0=y0,xAm=xAm,yAm=yAm,Jparameters=Jparameters,Randgain=Randgain,gaverage=gaverage,etaseries=etaseries,etaset=etaset,signeta=signeta,eigvalseries=eigvalseries[:,:,0],Reigvecseries=Reigvecseries,Leigvecseries=Leigvecseries,Beigvseries=Beigvseries[:,:,0],meigvecAm=meigvecAm[:,0],neigvecAm=neigvecAm[:,0],eigvAm=eigvAm,statesReig=statesReig,statesLeig=statesLeig,coveigvecseries=coveigvecseries,gYbasexee=gYbasexee,coeffeta=coeffeta)


* EIGENSPECTRA (eigenbulk and eigenvalue outlier)

In [ ]:
'''
with symmetry, 
eigvJ0
'''
idxgavg,idxtrial=9,8 # 6,3,0
figtspt,axtspt=plt.subplots(figsize=(6,3))
cm='bgc'
idxc=0
axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,1:]),np.imag(Beigvseries[idxgavg,idxtrial,1:]),s=5,c=cm[idxc],alpha=0.25) # >>>>>>>>>>>>>>
axtspt.scatter(np.real(Beigvseries[idxgavg,:,0]),np.imag(Beigvseries[idxgavg,:,0])-0.2*idxc,s=20,c=cm[idxc],marker='^',alpha=0.5) # >>>>>>>>>>>>
axtspt.set_aspect('equal')
axtspt.plot(np.ones(2)*eigvAm[0],np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
axtspt.spines['right'].set_color('none')
axtspt.spines['top'].set_color('none')
# X axis location
axtspt.xaxis.set_ticks_position('bottom')
axtspt.spines['bottom'].set_position(('data', 0))

aee,aei,aie,aii=xee,xei,xie,xii
print(xee,xei,xie,xii)
idxtrial=10
eta=0
xticks = np.linspace(-1.3,1.3,2)
xlims = [-1.3,1.3]
yticks = np.linspace(-1.3,1.3,2)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
# first do not multiply at
ahomo=gaverage
xee_,xei_,xie_,xii_=ahomo*aee/np.sqrt(2),ahomo*aei/np.sqrt(2),ahomo*aie/np.sqrt(2),ahomo*aii/np.sqrt(2)
gmat = np.array([[xee_**2,xei_**2],[xie_**2,xii_**2]])
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
r_g = np.sqrt(r_g2)
print(r_g)
## >>> Now we have symmetry >>
eta=etaseries[idxgavg]
longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
xr = longaxis*np.cos(theta)
yr = shortaxis*np.sin(theta)
axtspt.plot(xr, yr, color="gray", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>
axtspt.plot(longaxis*np.ones(2), np.array([-1.0,1.0]), color="gray", linewidth=2.5,linestyle='--') # >>>>>
axtspt.set_title(r'$\eta=$'+np.str(eta))
xticks = np.linspace(-3.5,2.5,2)
xlims = [-3.5,2.5]
yticks = np.linspace(-1.0,1.0,2)
ylims = [-1.1,1.1]
axtspt.set_xlim(xlims)
axtspt.set_ylim(ylims)
axtspt.set_xticks(xticks)
axtspt.set_yticks(yticks)
axtspt.set_aspect('equal')

# axtspt.plot(xr, yr, color="pink", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>
# axtspt.plot(longaxis*np.ones(2), np.array([-1.0,1.0]), color="pink", linewidth=2.5,linestyle='--') # >>>>>
# axtspt.set_title(r'$\eta=$'+np.str(eta))
# xticks = np.linspace(-3.5,2.5,2)
# xlims = [-3.5,2.5]
# yticks = np.linspace(-1.0,1.0,2)
# ylims = [-1.1,1.1]
# axtspt.set_xlim(xlims)
# axtspt.set_ylim(ylims)
# axtspt.set_xticks(xticks)
# axtspt.set_yticks(yticks)
# axtspt.set_aspect('equal')

* Population statistics(for backup)
    * eigenvalue outlier 
    * overlap between left and right eigenvectors

In [ ]:
'''
determinant lemma, odd terms 
even order terms should be maitained
'''
xticks = np.linspace(-2.91,-2.5,2)
xlims = [-2.91,-2.5]
yticks = np.linspace(-2.91,-2.5,2)
ylims = [-2.91,-2.5]

coeff=coeffeta.copy()
nAm=np.reshape(yAm*eigvAm[0],(NE*2,1))
mAm=np.reshape(xAm.copy(),(NE*2,1))
nAm,mAm=np.real(nAm),np.real(mAm)
y0E,y0I=np.mean(y0[:NE,0]),np.mean(y0[NE:,0])
x0E,x0I=np.mean(x0[:NE,0]),np.mean(x0[NE:,0])
lambda_theo2 = np.zeros(neta)
lambda_theo4 = np.zeros(neta)
lambda_num = np.mean(np.squeeze(Beigvseries[:,:,0]),axis=1)
cutoff = 2

for idxeta,eta in enumerate(etaseries):
    etaE,etaB,etaI=eta*coeff[0],eta*coeff[1],eta*coeff[2]
    '''
    Even-order terms have basic element XX.T(X.TX)
    '''
    bXXT=np.eye(NE*2)
    bXXT[:NE,:NE]*=(etaE*signeta[0]+etaB*signeta[1])/4
    bXXT[NE:,NE:]*=(etaI*signeta[2]+etaB*signeta[1])/4
    # cut off = 2
    C=comb(2,1)
    print(C)
    B2=C*(nAm.T@la.matrix_power(bXXT,1)@mAm)*gaverage**2. ## gaverage to the 2
    coeff2 = [1,-eigvJ0[0],0,-B2]
    C=comb(4,2)
    B4=C*(nAm.T@la.matrix_power(bXXT,2)@mAm)*gaverage**4 ## gaverage to the 4
    # B4=(nAm.T@(Y_t@YT_t@Y_t@YT_t)@mAm)*fory**2*foryt**2*6
    coeff4 = [1,-eigvJ0[0],0,-B2,0,-B4]
    # B2=(bcoeffE0*y0E*x0E*NE)+(bcoeffI0*y0I*x0I*NE)
    # coeff2 = [1,-eigvJ0[0],0,-B2*eigvJ0[0]]
    # B4=(bcoeffE0**2*y0E*x0E*NE)+(bcoeffI0**2*y0I*x0I*NE)
    # coeff4 = [1,-eigvJ0[0],0,-B2*eigvJ0[0],0,-B4*eigvJ0[0]]#(eta*eigvAm[0]*gaverage])
    lambda_test2=np.roots(coeff2)
    lambda_theo2[idxeta]=lambda_test2[0]
    lambda_test4=np.roots(coeff4)
    lambda_theo4[idxeta]=lambda_test4[0]
    print(lambda_test2[0],lambda_test4[0])

## >>>>>> eigenvalue outlier theo V.S. num    
fig,ax=plt.subplots(figsize=(5,5))  
ax.plot(lambda_num,lambda_theo2,'b',label='2 cut off')
ax.plot(lambda_num,lambda_theo4,'r',label='4 cut off')
ax.plot(lambda_num,lambda_num,'k',label='real')
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')
ax.legend()

## >>>>> eigenvalue outlier V.S. degree of symmetry
fig,ax=plt.subplots(figsize=(5,3))  
ax.plot(etaseries,lambda_num,'b',label='2 cut off')
ax.set_aspect('equal')

* First attemp, overlap/covariance between -- direction of change

In [ ]:
'''
variance of r and variance of l <r.Tr>, <l.T,l>
overlap between r and l <l.Tr>
'''
xticks = np.linspace(-0.01,0.1,2)
xlims = [-0.01,0.1]
yticks = np.linspace(-0.01,0.1,2)
ylims = [-0.01,0.1]


itrial=14
gY=np.squeeze(gYbasexee[itrial,:,:])
## >>>>> in the case with symmetry, the first eta point corresponds to eta = 0, which is just the original random matrix without symmetry
x0,y0=np.reshape(Reigvecseries[0,itrial,:,0],(NE*2,1)),np.reshape(Leigvecseries[0,itrial,:,0],(NE*2,1))

'''
rho_prime=eta/(2*sqrt(1-eta**2))
rho'/2/sqrt(rho) to 2nd=(1+np.sqrt(1-etaseries**2))/8/(1-etaseries**2)
rho'/2/sqrt(1-rho) to 2nd=(1-np.sqrt(1-etaseries**2))/8/(1-etaseries**2)
'''
heta=etaseries[1]-etaseries[0]
rhoseries=(1-np.sqrt(1-etaseries**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
sqrt_1meta2= np.sqrt(1-etaseries**2)
coeffsqrtY,coeffsqrtYT = (1-sqrt_1meta2)/sqrt_1meta2**2/8,(1+sqrt_1meta2)/sqrt_1meta2**2/8
overlapperturb,overlapvector = np.zeros((neta,2,2,NE),dtype=complex),np.zeros((neta,2,2,NE),dtype=complex)
# EI,LR,:
for i in range(1,neta-1):
    Jprimemat=np.sqrt(coeffsqrtYT[i])*gY.copy().T-np.sqrt(coeffsqrtY[i])*gY
    leftperturb=np.squeeze(-np.squeeze(y0.T@Jprimemat))/(-eigvAm[0])# left perturbation E
    rightperturb=np.squeeze(-np.squeeze(Jprimemat@x0))/(-eigvAm[0])#*Beigvseries[i,itrial,0]# right perturbation E
    overlapperturb[i,0,0,:],overlapperturb[i,0,1,:]=leftperturb[:NE],rightperturb[:NE]
    overlapperturb[i,1,0,:],overlapperturb[i,1,1,:]=leftperturb[NE:],rightperturb[NE:]
    overlapvector[i,0,0,:]=np.squeeze(Leigvecseries[i,itrial,:NE,0])-np.mean(np.squeeze(Leigvecseries[i,itrial,:NE,0]))
    overlapvector[i,1,0,:]=np.squeeze(Leigvecseries[i,itrial,NE:,0])-np.mean(np.squeeze(Leigvecseries[i,itrial,NE:,0]))
    overlapvector[i,0,1,:]=np.squeeze(Reigvecseries[i,itrial,:NE,0])-np.mean(np.squeeze(Reigvecseries[i,itrial,:NE,0]))
    overlapvector[i,1,1,:]=np.squeeze(Reigvecseries[i,itrial,NE:,0])-np.mean(np.squeeze(Reigvecseries[i,itrial,NE:,0]))
## E population 
# vectorcovE,perturbationcovE = np.zeros((neta,2,2)),np.zeros((neta,2,2))
directionvecE,directionvecI,directionperturbE,directionperturbI=np.zeros((neta,2,2)),np.zeros((neta,2,2)),np.zeros((neta,2,2)),np.zeros((neta,2,2))
ampvecE,ampvecI,ampperturbE,ampperturbI=np.zeros((neta,2)),np.zeros((neta,2)),np.zeros((neta,2)),np.zeros((neta,2))
for i in range(neta):
    vectorEmat,vectorImat=np.squeeze(overlapvector[i,0,:,:]),np.squeeze(overlapvector[i,1,:,:])
    perturbEmat,perturbImat=np.squeeze(overlapperturb[i,0,:,:]),np.squeeze(overlapperturb[i,1,:,:])
    vcovE,vcovI=vectorEmat@vectorEmat.T,vectorImat@vectorImat.T
    pcovE,pcovI=perturbEmat@perturbEmat.T ,perturbImat@perturbImat.T

    vvE,dirvE=la.eig(vcovE)
    vvI,dirvI=la.eig(vcovI)
    vpE,dirpE=la.eig(pcovE)
    vpI,dirpI=la.eig(pcovI)
    directionvecE[i,:,:],directionvecI[i,:,:]=dirvE,dirvI
    directionperturbE[i,:,:],directionperturbI[i,:,:]=dirpE,dirpI
    ampvecE[i,:],ampvecI[i,:]=vvE,vvI
    ampperturbE[i,:],ampperturbI[i,:]=vpE,vpI

# confidence_ellipse(Leftperturb,Rightperturb,ax,edgecolor='orange')
# confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor='green')

In [ ]:

fig,ax=plt.subplots(figsize=(5,5)) 
for idxeta in np.arange(1,8):
    heta=0.1
    ax.plot([0,0+ampvecE[idxeta,0]*directionvecE[idxeta,0,0]],[0,0+ampvecE[idxeta,0]*directionvecE[idxeta,1,0]],color='r',linestyle='-',linewidth=1.5)
    ax.plot([0,0+ampvecI[idxeta,0]*directionvecI[idxeta,0,0]],[0,0+ampvecI[idxeta,0]*directionvecI[idxeta,1,0]],color='orange',linestyle='-',linewidth=1.5)
    # ax.plot([0,0+ampvecE[idxeta+1,0]*directionvecE[idxeta+1,0,0]],[0,0+ampvecE[idxeta+1,0]*directionvecE[idxeta+1,1,0]],color='r',linestyle='-',linewidth=1.5)
    # ax.plot([0,0+ampvecI[idxeta+1,0]*directionvecI[idxeta+1,0,0]],[0,0+ampvecI[idxeta+1,0]*directionvecI[idxeta+1,1,0]],color='orange',linestyle='-',linewidth=1.5)

    vecEx,vecEy=ampvecE[idxeta,0]*directionvecE[idxeta,0,0],ampvecE[idxeta,0]*directionvecE[idxeta,1,0]
    vecIx,vecIy=ampvecI[idxeta,0]*directionvecI[idxeta,0,0],ampvecI[idxeta,0]*directionvecI[idxeta,1,0]
    ax.plot([vecEx,vecEx+ampperturbE[idxeta,1]*directionperturbE[idxeta,0,1]*heta],[vecEy,vecEy+ampperturbE[idxeta,1]*directionperturbE[idxeta,1,1]*heta],color='b',linestyle='--',linewidth=1.5)
    ax.plot([vecIx,vecIx+ampperturbI[idxeta,1]*directionperturbI[idxeta,0,1]*heta],[vecIy,vecIy+ampperturbI[idxeta,1]*directionperturbI[idxeta,1,1]*heta],color='g',linestyle='--',linewidth=1.5)

xticks = np.linspace(-0.0,0.15,2)
xlims = [-0.00,0.15]
yticks = np.linspace(-0.0,0.035,2)
ylims = [-0.0,0.025]
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
# ax.set_aspect('equal')



# fig,ax=plt.subplots(figsize=(5,5))  
# ax.scatter(overlapperturb[idxeta,0,0,:],overlapperturb[idxeta,0,1,:],s=5,c='orange',alpha=0.5)
# ax.scatter(Leigvecseries[idxeta,itrial,:NE,0]-np.mean(Leigvecseries[idxeta,itrial,:NE,0]),Reigvecseries[idxeta,itrial,:NE,0]-np.mean(Reigvecseries[idxeta,itrial,:NE,0]),s=5,c='blue',alpha=0.5)
# xticks = np.linspace(-0.06,0.06,2)
# xlims = [-0.06,0.06]
# yticks = np.linspace(-0.06,0.06,2)
# ylims = [-0.06,0.06]
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')

# fig,ax=plt.subplots(figsize=(5,5))  
# ax.scatter(overlapperturb[idxeta,1,0,:],overlapperturb[idxeta,1,1,:],s=5,c='orange',alpha=0.5)
# ax.scatter(Leigvecseries[idxeta,itrial,NE:,0]-np.mean(Leigvecseries[idxeta,itrial,NE:,0]),Reigvecseries[idxeta,itrial,NE:,0]-np.mean(Reigvecseries[idxeta,itrial,NE:,0]),s=5,c='blue',alpha=0.5)
# ax.set_aspect('equal')
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')

h=etaseries[1]-etaseries[0]
fig,ax=plt.subplots(figsize=(5,5))  
ax.scatter(overlapperturb[idxeta,0,0,:]*h,Leigvecseries[idxeta,itrial,:NE,0]-(Leigvecseries[idxeta-1,itrial,:NE,0]),s=5,c='orange',alpha=0.5)
ax.scatter(overlapperturb[idxeta,1,0,:]*h,Leigvecseries[idxeta,itrial,NE:,0]-(Leigvecseries[idxeta-1,itrial,NE:,0]),s=5,c='blue',alpha=0.5)
ax.set_aspect('equal')

fig,ax=plt.subplots(figsize=(5,5))  
ax.scatter(overlapperturb[idxeta,0,1,:]*h,Reigvecseries[idxeta,itrial,:NE,0]-(Reigvecseries[idxeta-1,itrial,:NE,0]),s=5,c='orange',alpha=0.5)
ax.scatter(overlapperturb[idxeta,1,1,:]*h,Reigvecseries[idxeta,itrial,NE:,0]-(Reigvecseries[idxeta-1,itrial,NE:,0]),s=5,c='blue',alpha=0.5)
ax.set_aspect('equal')

## For the overall process
* Choose a homogeneous $g_{\mathrm{avg}}$, choose a degree of symmetry $\eta$
    * from unperturbed connectivity (two points)
    * to perturbed connectivity without symmetry (no covariance)
    * finally the perturbed connectivity with. symmetry (covariance between eigenvector perturbations)

In [ ]:
'''
the whole changing process
from unperturbed connectivity to perturbed connectivity without symmetry to perturbed connectivity with symmetry
### >>>>>>>>> something to be noted 
    * use identical 
'''
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
nrank,ntrial,neta,ngavg=1,20,1,1
## for all casesq
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)

''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

'''
## parameters for unperturbed connectivity
'''
JI,JE,a,b=4.50,0.5,-0.6,0.6 # already recording -3.5# 2.0
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
'''
Am -- J(g0=0), l0(g0=0), r0(g0=0), S=R1(B1-lambda0In-1)^(-1)L1.T
'''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(1,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])
'''
for reference
'''
xAm,yAm=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))
'''
inverse S
'''
invB1=np.zeros((NE*2-1,NE*2-1))
for i in range(NE*2-1):
    invB1[i,i]=1/(eigvAm[i+1]-eigvAm[0])
X1,Y1=meigvecAm[:,1:].copy(),neigvecAm[:,1:].copy()
SAm = X1@invB1@Y1.T

'''
Network Setting for random gain 
'''
signeta=np.ones(3)
# signeta[1]*=(-1)
## heterogeneous random gain
'''
>>>>>>>>>>>>>> Case 1 and Case 2 >>>>>>>>>>>>>>>>>>>>>>>
'''
xee,xei,xie,xii=1.0,1.0,1.0,1.0 ## HOMOgeneous random gain g_kl
coeffeta=np.array([1.0,1.0,1.0])
'''
Recording Variables
Note>>>>>
    * the first '2' in each ndarray means the first stage -- without symmetry and the second stage -- with symmetry
    * the same trial share the same gY -- in the case with symmetric conn. gY and gY.T
'''
# original data
sampleidx=3
eigvalseries,mvecseries,nvecseries=np.zeros((sampleidx,ntrial,nrank),dtype=complex),np.zeros((sampleidx,ntrial,NE*2,nrank),dtype=complex),np.zeros((sampleidx,ntrial,NE*2,nrank),dtype=complex)
Reigvecseries,Leigvecseries=np.zeros((sampleidx,ntrial,NE*2,N),dtype=complex),np.zeros((sampleidx,ntrial,NE*2,N),dtype=complex)
Beigvseries=np.zeros((sampleidx,ntrial,N),dtype=complex)
''' All recorded connectivity statistics '''
statesReig,statesLeig=np.zeros((sampleidx,ntrial,3,2),dtype=complex),np.zeros((sampleidx,ntrial,3,2),dtype=complex) #E,I,Total/mean,std# 
coveigvecseries=np.zeros((sampleidx,ntrial,3),dtype=complex) # E, I and total
''' All random matrices for each trials '''
## Not X total (symmetry), but Y independent
gYbasexee=np.zeros((ntrial,NE*2,NE*2))
'''
for each individual trial, supposed to continuously increase g from zero to one -- see the sensibility changing
but within one single trial, the random matrix (also random structure) keeps unchanged, thus avoid changing in random pattern
'''

''' Iterative Processing '''
## >>>>>>>>>>>>>. CHOOSE PARAMETERS >>>>>>>>>>>>>>>>>
gaverage,eta=1.0,0.5
for iktrial in range(ntrial): 
    # >>> generate the (base/ref) random matrix for each trial
    Xsym=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
    Xsym[:NE,:NE]=xee*gaverage*np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(2*NE)
    Xsym[NE:,NE:]=xii*gaverage*np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(2*NE)
    Xsym[NE:,:NE]=xie*gaverage*np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(2*NE)
    Xsym[:NE,NE:]=xei*gaverage*np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(2*NE)
    XsymT=Xsym.copy().T

    ''' >>>>>>>>>>>>>>>>> without symmetry >>>>>>>>>>  '''
    X=Xsym.copy()
    idxeta = 0 ## the first stage -- without symmetry
    # overall
    J = X.copy()+Am.copy()
    JT = J.copy().T
    symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
    symnumfull*=2*NE
    print('eta full:',symtheofull,'; num (should be zero):',symnumfull)
    '''
    statistics: Full connectivity
    '''
    # properties
    eigvJ,eigrvecJ=la.eig(J)
    inveigrvecJ=la.inv(eigrvecJ)
    meig = np.squeeze(eigrvecJ[:,:].copy())
    neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
    neig=neig.copy().T
    for i in range(nrank):
        neig[:,i]*=eigvJ[i]
    for j in range(nrank,N):
        neig[:,j]*=eigvJ[j]
    ''' low rank properties m,n,eigenvalues '''
    eigvalseries[idxeta,iktrial,0],mvecseries[idxeta,iktrial,:,0],nvecseries[idxeta,iktrial,:,0]=eigvJ[0],meig[:,0].copy(),neig[:,0].copy() 
    
    Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
    ''' The first '''
    Reig=meig.copy()
    normval=np.sum(Reig*neig.copy(),axis=0)
    normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
    ''' eigendecomposition properties left and right eigenvectors '''
    Leig=neig.copy()/normval.copy() 
    '''
    normalization 0 (reference)
    '''
    tildex,tildey=np.reshape(Reig[:,0].copy(),(NE*2,1)),np.reshape(Leig[:,0].copy(),(NE*2,1))
    if (np.mean(tildex[:NE])*xAm[0,0])<0:
        tildex*=(-1)
        tildey*=(-1)
    xnorm0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
    ynorm0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey
    ## >>>>>>> '0' --> for the first stage without symmetr
    Reigvecseries[idxeta,iktrial,:,:],Leigvecseries[idxeta,iktrial,:,:]=Reig.copy(),Leig.copy()
    Beigvseries[idxeta,iktrial,:]=eigvJ.copy()
    Reigvecseries[idxeta,iktrial,:,0],Leigvecseries[idxeta,iktrial,:,0]=xnorm0[:,0].copy(),ynorm0[:,0].copy()
    '''  RECORDING STATES 
    * statesReig/Leig[grandom,trial,E/I/Total,mean/std]

    '''
    statesReig[idxeta,iktrial,0,0],statesReig[idxeta,iktrial,1,0]=np.mean(Reigvecseries[idxeta,iktrial,:NE,0]),np.mean(Reigvecseries[idxeta,iktrial,NE:,0])
    statesReig[idxeta,iktrial,2,0]=np.mean(Reigvecseries[idxeta,iktrial,:,0])
    statesReig[idxeta,iktrial,0,1],statesReig[idxeta,iktrial,1,1]=np.std(Reigvecseries[idxeta,iktrial,:NE,0]),np.std(Reigvecseries[idxeta,iktrial,NE:,0])
    statesReig[idxeta,iktrial,2,1]=np.std(Reigvecseries[idxeta,iktrial,:,0])
    
    statesLeig[idxeta,iktrial,0,0],statesLeig[idxeta,iktrial,1,0]=np.mean(Leigvecseries[idxeta,iktrial,:NE,0]),np.mean(Leigvecseries[idxeta,iktrial,NE:,0])
    statesLeig[idxeta,iktrial,2,0]=np.mean(Leigvecseries[idxeta,iktrial,:,0])
    statesLeig[idxeta,iktrial,0,1],statesLeig[idxeta,iktrial,1,1]=np.std(Leigvecseries[idxeta,iktrial,:NE,0]),np.std(Leigvecseries[idxeta,iktrial,NE:,0])
    statesLeig[idxeta,iktrial,2,1]=np.std(Leigvecseries[idxeta,iktrial,:,0])
    ''' covariance '''
    '''
    Note, we should change the normalization for the first item to the (Refer) normalization 
    tildeReig,tildeLeig
    normalization 0
    '''
    ## normalization 0 reference
    tildeReig,tildeLeig=np.squeeze(Reigvecseries[idxeta,iktrial,:,:]),np.squeeze(Leigvecseries[idxeta,iktrial,:,:])
    reig,leig=np.reshape(tildeReig[:,0].copy(),(NE*2,1)),np.reshape(tildeLeig[:,0].copy(),(NE*2,1))
    ## mean at eact point
    rEceneig,rIceneig = np.mean(reig[:NE,0]),np.mean(reig[NE:,0])
    lEceneig,lIceneig = np.mean(leig[:NE,0]),np.mean(leig[NE:,0])
    ## random components
    rrandeig,lrandeig=reig.copy(),leig.copy()
    rrandeig[:NE,0],rrandeig[NE:,0]=rrandeig[:NE,0]-rEceneig,rrandeig[NE:,0]-rIceneig
    lrandeig[:NE,0],lrandeig[NE:,0]=lrandeig[:NE,0]-lEceneig,lrandeig[NE:,0]-lIceneig

    coveigvecseries[idxeta,iktrial,0]=np.reshape(rrandeig[:NE,0],(NE,1)).T@np.reshape(lrandeig[:NE,0],(NE,1))
    coveigvecseries[idxeta,iktrial,1]=np.reshape(rrandeig[NE:,0],(NE,1)).T@np.reshape(lrandeig[NE:,0],(NE,1))
    coveigvecseries[idxeta,iktrial,2]=coveigvecseries[idxeta,iktrial,0]+coveigvecseries[idxeta,iktrial,1]

    '''
    converting initial x0 and y0 to the one with random components
    '''
    x0,y0=np.reshape(np.squeeze(Reigvecseries[idxeta,iktrial,:,0]),(NE*2,1)),np.reshape(np.squeeze(Leigvecseries[idxeta,iktrial,:,0]),(2*NE,1)) 
    ## >>>>>>>>>>  with g but without symmetry

    ''' >>>>>>>>>>>>>>>>> with symmetry >>>>>>>>>>  '''
    for idxeta in range(1,3,1):
        # etaset[0]*=0.1
        Xinit = Xsym.copy()
        # idxeta=1
        etaset=idxeta*eta*np.ones(3)
        etaset=etaset*coeffeta
        print('a set of symmetry:',etaset)
        ''' heterogeneous ETA method 1 '''
        ## EE ##
        asqr=(1-np.sqrt(1-etaset[0]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[:NE,:NE]=signeta[0]*aamp*XsymT[:NE,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,:NE].copy()
        ## EI IE##
        asqr=(1-np.sqrt(1-etaset[1]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,:NE]=signeta[1]*aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        ## II ##
        asqr=(1-np.sqrt(1-etaset[2]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,NE:]=signeta[2]*aamp*XsymT[NE:,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,NE:].copy()
        ''' THEORETICAL FULL ETA '''
        symtheofull=gaverage**2*(etaset[0]*signeta[0]+etaset[2]*signeta[2]+2*etaset[1]*signeta[1])/4.0#+(a-b)**2/(NE*2)
        X=Xinit.copy()

        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
        symnumfull*=2*NE
        print('compare degree of symmetry:',symnumfull,'--num; ',etaset*signeta,'--setting')
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:].copy())
        # meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
        neig=neig.copy().T
        print('test normal 4:',np.sum(neig[:,0]*meig[:,0]))
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
        for j in range(1,N):
            neig[:,j]*=eigvJ[j]
        ## >>> for low rank framework m and n
        eigvalseries[idxeta,iktrial,0],mvecseries[idxeta,iktrial,:,0],nvecseries[idxeta,iktrial,:,0]=eigvJ[0],meig[:,0].copy(),neig[:,0].copy()
        
        ## >>> for eigendecomposition 
        Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
        ''' The first '''
        Reig=meig.copy()
        normval=np.sum(Reig*neig.copy(),axis=0)
        normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        Leig=neig.copy()/normval.copy()
        print('test normal 5:',np.sum(Leig[:,0]*Reig[:,0]))
        '''
        normalization 0 (reference)
        '''
        tildex,tildey=Reig[:,0].copy(),Leig[:,0].copy()
        if (np.mean(tildex[:NE])*xAm[0,0])<0:
            tildex*=(-1)
            tildey*=(-1)
        xnorm0=np.sqrt(np.squeeze(tildey.T@x0)/np.squeeze(y0.T@tildex))*tildex
        ynorm0=np.sqrt(np.squeeze(tildex.T@y0)/np.squeeze(x0.T@tildey))*tildey

        Reigvecseries[idxeta,iktrial,:,:],Leigvecseries[idxeta,iktrial,:,:]=Reig.copy(),Leig.copy()
        Beigvseries[idxeta,iktrial,:]=eigvJ.copy()
        Reigvecseries[idxeta,iktrial,:,0],Leigvecseries[idxeta,iktrial,:,0]=xnorm0[:].copy(),ynorm0[:].copy()

        '''  RECORDING STATES '''
        statesReig[idxeta,iktrial,0,0],statesReig[idxeta,iktrial,1,0]=np.mean(Reigvecseries[idxeta,iktrial,:NE,0]),np.mean(Reigvecseries[idxeta,iktrial,NE:,0])
        statesReig[idxeta,iktrial,2,0]=np.mean(Reigvecseries[idxeta,iktrial,:,0])
        statesReig[idxeta,iktrial,0,1],statesReig[idxeta,iktrial,1,1]=np.std(Reigvecseries[idxeta,iktrial,:NE,0]),np.std(Reigvecseries[idxeta,iktrial,NE:,0])
        statesReig[idxeta,iktrial,2,1]=np.std(Reigvecseries[idxeta,iktrial,:,0])
        
        statesLeig[idxeta,iktrial,0,0],statesLeig[idxeta,iktrial,1,0]=np.mean(Leigvecseries[idxeta,iktrial,:NE,0]),np.mean(Leigvecseries[idxeta,iktrial,NE:,0])
        statesLeig[idxeta,iktrial,2,0]=np.mean(Leigvecseries[idxeta,iktrial,:,0])
        statesLeig[idxeta,iktrial,0,1],statesLeig[idxeta,iktrial,1,1]=np.std(Leigvecseries[idxeta,iktrial,:NE,0]),np.std(Leigvecseries[idxeta,iktrial,NE:,0])
        statesLeig[idxeta,iktrial,2,1]=np.std(Leigvecseries[idxeta,iktrial,:,0])

        ''' covariance '''
        '''
        Note, we should change the normalization for the first item to the (Refer) normalization 
        tildeReig,tildeLeig
        normalization 0
        '''
        ## normalization 0 reference
        tildeReig,tildeLeig=np.squeeze(Reigvecseries[idxeta,iktrial,:,:]),np.squeeze(Leigvecseries[idxeta,iktrial,:,:])
        reig,leig=np.reshape(tildeReig[:,0].copy(),(NE*2,1)),np.reshape(tildeLeig[:,0].copy(),(NE*2,1))
        ## mean at eact point
        rEceneig,rIceneig = np.mean(reig[:NE,0]),np.mean(reig[NE:,0])
        lEceneig,lIceneig = np.mean(leig[:NE,0]),np.mean(leig[NE:,0])
        ## random components
        rrandeig,lrandeig=reig.copy(),leig.copy()
        rrandeig[:NE,0],rrandeig[NE:,0]=rrandeig[:NE,0]-rEceneig,rrandeig[NE:,0]-rIceneig
        lrandeig[:NE,0],lrandeig[NE:,0]=lrandeig[:NE,0]-lEceneig,lrandeig[NE:,0]-lIceneig

        coveigvecseries[idxeta,iktrial,0]=np.reshape(rrandeig[:NE,0],(NE,1)).T@np.reshape(lrandeig[:NE,0],(NE,1))
        coveigvecseries[idxeta,iktrial,1]=np.reshape(rrandeig[NE:,0],(NE,1)).T@np.reshape(lrandeig[NE:,0],(NE,1))
        coveigvecseries[idxeta,iktrial,2]=coveigvecseries[idxeta,iktrial,0]+coveigvecseries[idxeta,iktrial,1]
    gYbasexee[iktrial,:,:]=Xsym.copy()


In [ ]:
* Recording PARAMETERS, VARIABLES and DATA

In [ ]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE
Randgain=np.zeros(4)
Randgain[0],Randgain[1],Randgain[2],Randgain[3]=xee,xei,xie,xii

np.savez(now+'_perturbationConvert_data',x0=x0,y0=y0,xAm=xAm,yAm=yAm,Jparameters=Jparameters,Randgain=Randgain,gaverage=gaverage,eta=eta,signeta=signeta,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries[:,:,:,0],Leigvecseries=Leigvecseries[:,:,:,0],Beigvseries=Beigvseries[:,:,0],meigvecAm=meigvecAm[:,0],neigvecAm=neigvecAm[:,0],eigvAm=eigvAm,statesReig=statesReig,statesLeig=statesLeig,coveigvecseries=coveigvecseries,gYbasexee=gYbasexee,coeffeta=coeffeta)


* Based on ensemble average, for the loading patterns

In [ ]:
'''
based on ensemble-averaging, ntrial * 
'''
## for all casesq
xticks = np.linspace(-0.1,0.06,2)
xlims = [-0.1,0.06]
yticks = np.linspace(-0.05,0.01,2)
ylims = [-0.05,0.01]

## >>>>>>>>>> First, two points >>>>>>>>>>>>>>>>.
figa,axa=plt.subplots(figsize=(5,3))
axa.scatter(yAm[:NE,0],xAm[:NE,0],s=30.0,c='black',marker='^',alpha=1.0)
axa.scatter(yAm[NE:,0],xAm[NE:,0],s=30.0,c='black',marker='^',alpha=1.0)
axa.axvline(c='grey', lw=1)
axa.axhline(c='grey', lw=1)
axa.set_xlim(xlims)
axa.set_ylim(ylims)
axa.set_xticks(xticks)
axa.set_yticks(yticks)
axa.set_aspect('equal')

nsymsetE,nsymsetI,symsetE,symsetI=np.zeros((2,ntrial*NE)),np.zeros((2,ntrial*NE)),np.zeros((2,ntrial*NE)),np.zeros((2,ntrial*NE))
symsetEp,symsetIp=np.zeros((2,ntrial*NE)),np.zeros((2,ntrial*NE))
nsymsetE[0,:],nsymsetE[1,:]=np.squeeze(Leigvecseries[0,:,:NE,0]).flatten(),np.squeeze(Reigvecseries[0,:,:NE,0]).flatten()
nsymsetI[0,:],nsymsetI[1,:]=np.squeeze(Leigvecseries[0,:,NE:,0]).flatten(),np.squeeze(Reigvecseries[0,:,NE:,0]).flatten()
symsetE[0,:],symsetE[1,:]=np.squeeze(Leigvecseries[1,:,:NE,0]).flatten(),np.squeeze(Reigvecseries[1,:,:NE,0]).flatten()
symsetI[0,:],symsetI[1,:]=np.squeeze(Leigvecseries[1,:,NE:,0]).flatten(),np.squeeze(Reigvecseries[1,:,NE:,0]).flatten()

symsetEp[0,:],symsetEp[1,:]=np.squeeze(Leigvecseries[2,:,:NE,0]).flatten(),np.squeeze(Reigvecseries[2,:,:NE,0]).flatten()
symsetIp[0,:],symsetIp[1,:]=np.squeeze(Leigvecseries[2,:,NE:,0]).flatten(),np.squeeze(Reigvecseries[2,:,NE:,0]).flatten()

###  CALCULATE MEAN AND RANDOM 
mnsymE,mnsymI = np.reshape(np.mean(nsymsetE,axis=1),(2,1)),np.reshape(np.mean(nsymsetI,axis=1),(2,1))
msymE,msymI   = np.reshape(np.mean(symsetE,axis=1),(2,1)),np.reshape(np.mean(symsetI,axis=1),(2,1))
rnsymE,rnsymI = nsymsetE-np.repeat(mnsymE,NE*ntrial,axis=1),nsymsetI-np.repeat(mnsymI,NE*ntrial,axis=1)
rsymE,rsymI = symsetE-np.repeat(msymE,NE*ntrial,axis=1),symsetI-np.repeat(msymI,NE*ntrial,axis=1)

msymEp,msymIp   = np.reshape(np.mean(symsetEp,axis=1),(2,1)),np.reshape(np.mean(symsetIp,axis=1),(2,1))
rsymEp,rsymIp = symsetEp-np.repeat(msymEp,NE*ntrial,axis=1),symsetIp-np.repeat(msymIp,NE*ntrial,axis=1)

### CALCULATE COVARIANCE
covnsymE,covnsymI = rnsymE@rnsymE.T,rnsymI@rnsymI.T
covsymE,covsymI   = rsymE@rsymE.T,rsymI@rsymI.T 
covsymEp,covsymIp   = rsymEp@rsymEp.T,rsymIp@rsymIp.T 
### DIRECTIONS AND EIGENVALUES(A,PLITUDES)
ampnsymE,dirnsymE = la.eig(covnsymE)
ampnsymI,dirnsymI = la.eig(covnsymI)
ampsymE,dirsymE   = la.eig(covsymE)
ampsymI,dirsymI   = la.eig(covsymI)
ampsymEp,dirsymEp   = la.eig(covsymEp)
ampsymIp,dirsymIp   = la.eig(covsymIp)

## >>>>>>>>>>>. second, without symmetry >>>>>>>>
figg,axg=plt.subplots(figsize=(5,3))  
cm='rb'
# edgv='gray'
edgv='black'
# cm='mc'
axg.scatter(nsymsetE[0,:],nsymsetE[1,:],s=5.0,c=cm[0],alpha=0.25)
axg.scatter(nsymsetI[0,:],nsymsetI[1,:],s=5.0,c=cm[1],alpha=0.25)
axg.plot([mnsymE[0],mnsymE[0]+dirnsymE[0,0]],[mnsymE[1],mnsymE[1]+dirnsymE[1,0]],color='grey',linestyle='--',linewidth=1.5)
axg.plot([mnsymE[0],mnsymE[0]+dirnsymE[0,1]],[mnsymE[1],mnsymE[1]+dirnsymE[1,1]],color='grey',linestyle='--',linewidth=1.5)
axg.plot([mnsymI[0],mnsymI[0]+dirnsymI[0,0]],[mnsymI[1],mnsymI[1]+dirnsymI[1,0]],color='grey',linestyle='--',linewidth=1.5)
axg.plot([mnsymI[0],mnsymI[0]+dirnsymI[0,1]],[mnsymI[1],mnsymI[1]+dirnsymI[1,1]],color='grey',linestyle='--',linewidth=1.5)
confidence_ellipse(nsymsetE[0,:],nsymsetE[1,:],axg,edgecolor=edgv)
confidence_ellipse(nsymsetI[0,:],nsymsetI[1,:],axg,edgecolor=edgv)
axg.axvline(c='grey', lw=1)
axg.axhline(c='grey', lw=1)
axg.set_xlim(xlims)
axg.set_ylim(ylims)
axg.set_xticks(xticks)
axg.set_yticks(yticks)
axg.set_aspect('equal')

## >>>>>>>>>>>. second, without symmetry >>>>>>>>
figs,axs=plt.subplots(figsize=(5,3))  
cm='rb'
# cm='mc'
edgv='black'
axs.scatter(symsetE[0,:],symsetE[1,:],s=5.0,c=cm[0],alpha=0.25)
axs.scatter(symsetI[0,:],symsetI[1,:],s=5.0,c=cm[1],alpha=0.25)
axs.plot([msymE[0],msymE[0]+dirsymE[0,0]],[msymE[1],msymE[1]+dirsymE[1,0]],color='grey',linestyle='-.',linewidth=1.5)
axs.plot([msymE[0],msymE[0]+dirsymE[0,1]],[msymE[1],msymE[1]+dirsymE[1,1]],color='grey',linestyle='-.',linewidth=1.5)
axs.plot([msymI[0],msymI[0]+dirsymI[0,0]],[msymI[1],msymI[1]+dirsymI[1,0]],color='grey',linestyle='-.',linewidth=1.5)
axs.plot([msymI[0],msymI[0]+dirsymI[0,1]],[msymI[1],msymI[1]+dirsymI[1,1]],color='grey',linestyle='-.',linewidth=1.5)
confidence_ellipse(symsetE[0,:],symsetE[1,:],axs,edgecolor=edgv)
confidence_ellipse(symsetI[0,:],symsetI[1,:],axs,edgecolor=edgv)

axs.axvline(c='grey', lw=1)
axs.axhline(c='grey', lw=1)
axs.set_xlim(xlims)
axs.set_ylim(ylims)
axs.set_xticks(xticks)
axs.set_yticks(yticks)
axs.set_aspect('equal')

## >>>>>>>>>>>. second, without symmetry >>>>>>>>
figsp,axsp=plt.subplots(figsize=(5,3))  
cm='rb'
# cm='mc'
edgv='black'
axsp.scatter(symsetEp[0,:],symsetEp[1,:],s=5.0,c=cm[0],alpha=0.25)
axsp.scatter(symsetIp[0,:],symsetIp[1,:],s=5.0,c=cm[1],alpha=0.25)
axsp.plot([msymEp[0],msymEp[0]+dirsymEp[0,0]],[msymEp[1],msymEp[1]+dirsymEp[1,0]],color='grey',linestyle='-.',linewidth=1.5)
axsp.plot([msymEp[0],msymEp[0]+dirsymEp[0,1]],[msymEp[1],msymEp[1]+dirsymEp[1,1]],color='grey',linestyle='-.',linewidth=1.5)
axsp.plot([msymIp[0],msymIp[0]+dirsymIp[0,0]],[msymIp[1],msymIp[1]+dirsymIp[1,0]],color='grey',linestyle='-.',linewidth=1.5)
axsp.plot([msymIp[0],msymIp[0]+dirsymIp[0,1]],[msymIp[1],msymIp[1]+dirsymIp[1,1]],color='grey',linestyle='-.',linewidth=1.5)
confidence_ellipse(symsetEp[0,:],symsetEp[1,:],axsp,edgecolor=edgv)
confidence_ellipse(symsetIp[0,:],symsetIp[1,:],axsp,edgecolor=edgv)

axsp.axvline(c='grey', lw=1)
axsp.axhline(c='grey', lw=1)
axsp.set_xlim(xlims)
axsp.set_ylim(ylims)
axsp.set_xticks(xticks)
axsp.set_yticks(yticks)
axsp.set_aspect('equal')

'''
Comparing together
'''
## >>>>>>>>>> First, two points >>>>>>>>>>>>>>>>.
fig0,ax0=plt.subplots(figsize=(5,3))
ax0.scatter(yAm[:NE,0],xAm[:NE,0],s=30.0,c='black',marker='^',alpha=1.0)
ax0.scatter(yAm[NE:,0],xAm[NE:,0],s=30.0,c='black',marker='^',alpha=1.0)

ax0.plot([mnsymE[0],mnsymE[0]+dirnsymE[0,0]],[mnsymE[1],mnsymE[1]+dirnsymE[1,0]],color='grey',linestyle='--',linewidth=1.5)
ax0.plot([mnsymE[0],mnsymE[0]+dirnsymE[0,1]],[mnsymE[1],mnsymE[1]+dirnsymE[1,1]],color='grey',linestyle='--',linewidth=1.5)
ax0.plot([mnsymI[0],mnsymI[0]+dirnsymI[0,0]],[mnsymI[1],mnsymI[1]+dirnsymI[1,0]],color='grey',linestyle='--',linewidth=1.5)
ax0.plot([mnsymI[0],mnsymI[0]+dirnsymI[0,1]],[mnsymI[1],mnsymI[1]+dirnsymI[1,1]],color='grey',linestyle='--',linewidth=1.5)
confidence_ellipse(nsymsetE[0,:],nsymsetE[1,:],ax0,edgecolor=edgv)
confidence_ellipse(nsymsetI[0,:],nsymsetI[1,:],ax0,edgecolor=edgv)

ax0.plot([msymE[0],msymE[0]+dirsymE[0,0]],[msymE[1],msymE[1]+dirsymE[1,0]],color='grey',linestyle='-.',linewidth=1.5)
axs.plot([msymE[0],msymE[0]+dirsymE[0,1]],[msymE[1],msymE[1]+dirsymE[1,1]],color='grey',linestyle='-.',linewidth=1.5)
ax0.plot([msymI[0],msymI[0]+dirsymI[0,0]],[msymI[1],msymI[1]+dirsymI[1,0]],color='grey',linestyle='-.',linewidth=1.5)
ax0.plot([msymI[0],msymI[0]+dirsymI[0,1]],[msymI[1],msymI[1]+dirsymI[1,1]],color='grey',linestyle='-.',linewidth=1.5)
confidence_ellipse(symsetE[0,:],symsetE[1,:],ax0,edgecolor=edgv)
confidence_ellipse(symsetI[0,:],symsetI[1,:],ax0,edgecolor=edgv)

ax0.axvline(c='grey', lw=1)
ax0.axhline(c='grey', lw=1)
ax0.set_xlim(xlims)
ax0.set_ylim(ylims)
ax0.set_xticks(xticks)
ax0.set_yticks(yticks)
ax0.set_aspect('equal')

In [ ]:
'''
>>>>>>>>>>>>>>>>> Compare the individual response (not statistics), NOTE:
To make direct comparison between numerical and perturbation theorem, keep the same random matrix for each trial
choose one trial -- determine the random derivative -- calculate the individual elements in vector

'''
xticks = np.linspace(-0.05,0.05,2)
xlims = [-0.05,0.05]
yticks = np.linspace(-0.05,0.05,2)
ylims = [-0.05,0.05]
'''
# CHOOSE ONE TRIAL
'''
shiftepsilon=0.02
idxtrial,idxtrial_=8,3
### x0,y0 change with idxtrial
x0,y0=Reigvecseries[0,idxtrial,:,0],Leigvecseries[0,idxtrial,:,0]
fig,ax=plt.subplots(2,2,figsize=(4,4))

gsamples=gaverage
rperturbE_num,lperturbE_num = np.zeros((NE*ntrial,2)),np.zeros((NE*ntrial,2))
rperturbE_the,lperturbE_the = np.zeros((NE*ntrial,2)),np.zeros((NE*ntrial,2))
rperturbI_num,lperturbI_num = np.zeros((NE*ntrial,2)),np.zeros((NE*ntrial,2))
rperturbI_the,lperturbI_the = np.zeros((NE*ntrial,2)),np.zeros((NE*ntrial,2))

rprime_the,lprime_the=np.zeros((ntrial,NE*2)),np.zeros((ntrial,NE*2))
for idxtrial in range(ntrial):
    Xprime = np.squeeze(gYbasexee[idxtrial,:,:])/gaverage
    rprime_the[idxtrial,:],lprime_the[idxtrial,:] = np.squeeze(Xprime@xAm/eigvAm[0]),np.squeeze(Xprime.T@yAm/eigvAm[0])
i=0 # without symmetry
lperturbE_num[:,i] = (np.squeeze(Leigvecseries[i,:,:NE,0])-np.repeat(np.reshape(yAm[:NE],(1,NE)),ntrial,axis=0)).flatten()
rperturbE_num[:,i] = (np.squeeze(Reigvecseries[i,:,:NE,0])-np.repeat(np.reshape(xAm[:NE],(1,NE)),ntrial,axis=0)).flatten()
lperturbE_the[:,i] = (lprime_the[:,:NE]*gaverage).flatten()
rperturbE_the[:,i] = (rprime_the[:,:NE]*gaverage).flatten()

lperturbI_num[:,i] = (np.squeeze(Leigvecseries[i,:,NE:,0])-np.repeat(np.reshape(yAm[NE:],(1,NE)),ntrial,axis=0)).flatten()
rperturbI_num[:,i] = (np.squeeze(Reigvecseries[i,:,NE:,0])-np.repeat(np.reshape(xAm[NE:],(1,NE)),ntrial,axis=0)).flatten()
lperturbI_the[:,i] = (lprime_the[:,NE:]*gaverage).flatten()
rperturbI_the[:,i] = (rprime_the[:,NE:]*gaverage).flatten()

ax[0,i].scatter(np.real(lperturbE_the[:,i]),rperturbE_num[:,i],s=5,c='black',alpha=0.25)

ax[1,i].scatter(np.real(lperturbI_the[:,i]),rperturbI_num[:,i],s=5,c='black',alpha=0.25)

ax[0,i].set_xticks([])
ax[0,i].set_yticks([])
ax[1,i].set_xticks([])
ax[1,i].set_yticks([])

'''
rho_prime=eta/(2*sqrt(1-eta**2))
rho'/2/sqrt(rho) to 2nd=(1+np.sqrt(1-etaseries**2))/8/(1-etaseries**2)
rho'/2/sqrt(1-rho) to 2nd=(1-np.sqrt(1-etaseries**2))/8/(1-etaseries**2)
'''
i=1 # with symmetry

dh=1
leftperturb,rightperturb=np.zeros((ntrial,NE*2)),np.zeros((ntrial,NE*2))
for idxtrial in range(ntrial):
    gY = np.squeeze(gYbasexee[idxtrial,:,:])
    rhoseries=(1-np.sqrt(1-(eta/1.0)**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
    sqrt_1meta2= np.sqrt(1-(eta/1.0)**2)
    coeffsqrtY,coeffsqrtYT = (1-sqrt_1meta2)/sqrt_1meta2**2/8,(1+sqrt_1meta2)/sqrt_1meta2**2/8
    # EI,LR,:
    Jprimemat=np.sqrt(coeffsqrtYT)*gY.copy().T-np.sqrt(coeffsqrtY)*gY
    leftperturb[idxtrial,:]=np.squeeze(-np.squeeze(y0.T@Jprimemat))/(-eigvAm[0])# left perturbation E
    rightperturb[idxtrial,:]=np.squeeze(-np.squeeze(Jprimemat@x0))/(-eigvAm[0])#*Beigvseries[i,itrial,0]# right perturbation E

## perturbed vector
lperturbE_the[:,i] = np.squeeze(leftperturb[:,:NE]).flatten()
rperturbE_the[:,i] = np.squeeze(rightperturb[:,:NE]).flatten()
## delta vector
lperturbE_num[:,i]=(np.squeeze(Leigvecseries[i,:,:NE,0])-np.squeeze(y0[:NE])).flatten()
rperturbE_num[:,i]=(np.squeeze(Reigvecseries[i,:,:NE,0])-np.squeeze(x0[:NE])).flatten()

## perturbed vector
lperturbI_the[:,i] = np.squeeze(leftperturb[:,NE:]).flatten()
rperturbI_the[:,i] = np.squeeze(rightperturb[:,NE:]).flatten()
## delta vector
lperturbI_num[:,i]=(np.squeeze(Leigvecseries[i,:,NE:,0])-np.squeeze(y0[NE:])).flatten()
rperturbI_num[:,i]=(np.squeeze(Reigvecseries[i,:,NE:,0])-np.squeeze(x0[NE:])).flatten()

ax[0,i].scatter(np.real(lperturbE_the[:,i]),rperturbE_the[:,i],s=5,c='black',alpha=0.25)
ax[1,i].scatter(np.real(lperturbI_the[:,i]),rperturbI_the[:,i],s=5,c='black',alpha=0.25)

ax[0,i].set_xticks([])
ax[0,i].set_yticks([])
ax[1,i].set_xticks([])
ax[1,i].set_yticks([])
ax[1,i].set_aspect('equal')

for j in range(2):
    for i in range(2):
        ax[j,i].set_xlim(xlims)
        ax[j,i].set_ylim(ylims)
        ax[j,i].axvline(c='grey', lw=1)
        ax[j,i].axhline(c='grey', lw=1)
ax[0,0].set_yticks(yticks)
ax[1,0].set_yticks(yticks)

# ## >>>>>>>>> direction >>>>>>
# ampnsymE,dirnsymE = la.eig(covnsymE)
# ampnsymI,dirnsymI = la.eig(covnsymI)
# ampsymE,dirsymE   = la.eig(covsymE)
# ampsymI,dirsymI   = la.eig(covsymI)
# ampsymEp,dirsymEp   = la.eig(covsymEp)
# ampsymIp,dirsymIp   = la.eig(covsymIp)

perturbEtheo,perturbItheo=np.zeros((2,NE*ntrial)),np.zeros((2,NE*ntrial))
perturbEtheo[0,:],perturbEtheo[1,:]=lperturbE_the[:,1],rperturbE_the[:,1]
perturbItheo[0,:],perturbItheo[1,:]=lperturbI_the[:,1],rperturbI_the[:,1]
covsymtheE,covsymtheI=perturbEtheo@perturbEtheo.T,perturbItheo@perturbItheo.T
ampsymtheE,dirsymtheE=la.eig(covsymtheE)
ampsymtheI,dirsymtheI=la.eig(covsymtheI)

# vecEx,vecEy=ampxy0E[0]*dirxy0E[0,0],ampxy0E[0]*dirxy0E[1,0]
ax[0,1].plot([0,0+ampsymtheE[1]*dirsymtheE[0,1]],[0,0+ampsymtheE[1]*dirsymtheE[1,1]],color='blue',linestyle='--',linewidth=1.5)
ax[1,1].plot([0,0+ampsymtheI[1]*dirsymtheI[0,1]],[0,0+ampsymtheI[1]*dirsymtheI[1,1]],color='blue',linestyle='--',linewidth=1.5)
ax[0,1].plot([0,0+ampsymtheE[0]*dirsymtheE[0,0]],[0,0+ampsymtheE[0]*dirsymtheE[1,0]],color='gray',linestyle='--',linewidth=1.5)
ax[1,1].plot([0,0+ampsymtheI[0]*dirsymtheI[0,0]],[0,0+ampsymtheI[0]*dirsymtheI[1,0]],color='gray',linestyle='--',linewidth=1.5)

ax[0,0].plot([0,0+ampnsymE[1]*dirnsymE[0,1]],[0,0+ampnsymE[1]*dirnsymE[1,1]],color='gray',linestyle='--',linewidth=1.5)
ax[1,0].plot([0,0+ampnsymI[1]*dirnsymI[0,1]],[0,0+ampnsymI[1]*dirnsymI[1,1]],color='gray',linestyle='--',linewidth=1.5)
ax[0,0].plot([0,0+ampnsymE[0]*dirnsymE[0,0]],[0,0+ampnsymE[0]*dirnsymE[1,0]],color='blue',linestyle='--',linewidth=1.5)
ax[1,0].plot([0,0+ampnsymI[0]*dirnsymI[0,0]],[0,0+ampnsymI[0]*dirnsymI[1,0]],color='blue',linestyle='--',linewidth=1.5)

## SUBCIRCUIT SYMMETRY (GENRE)

In [ ]:
'''
WITH SYMMETRY
'''
'''
Cases 3,4 with symmetry
Focus on mean 
Focus on second moment
Focus on derivative on 
'''
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
'''
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
'''
nrank,ntrial,neta,ngavg=1,10,10,10

## for all casesq
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)

''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

''' 
---------- method 2 ------------------ 
'''
''' ## Three \bar{J} cases '''
# JI,JE,a,b=3.5,1.0,0,0#1.0,3.5,0,0#3.5,1.00,-0.6,0.6 # already recording -3.5# 2.0
JI,JE,a,b=1.0,3.5,0,0#1.0,3.5,0,0#3.5,1.00,-0.6,0.6 # already recording -3.5# 2.0
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(1,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe
''' for reference '''
xAm,yAm=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))

'''
##>>>>>>>>>>> g0!=0, one example >>>>>>>>
'''
gaverage=1.0
# X0=np.random.randn(NE*2,NE*2)*gaverage/np.sqrt(2*NE) # /np.sqrt(2) no later
X0=np.zeros((NE*2,NE*2))
X0[:NE,:NE]=np.reshape(np.random.randn(NE*NE),(NE,NE))*gaverage/np.sqrt(2*NE)
X0[NE:,NE:]=np.reshape(np.random.randn(NE*NE),(NE,NE))*gaverage/np.sqrt(2*NE)
X0[NE:,:NE]=np.reshape(np.random.randn(NE*NE),(NE,NE))*gaverage/np.sqrt(2*NE)
X0[:NE,NE:]=np.reshape(np.random.randn(NE*NE),(NE,NE))*gaverage/np.sqrt(2*NE)
Xsym=X0.copy()
XsymT=Xsym.copy().T
J0=Am.copy()+X0.copy()
## >>>>>> calculate the original \bar{m} and \bar{n} 
eigvJ0,eigrvecJ0=la.eig(J0)
inveigrvecJ0=la.inv(eigrvecJ0)
meig0 = np.squeeze(eigrvecJ0[:,:].copy())
neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
neig0=neig0.copy().T
## test neig0*meig0=1
print('test normal:',np.sum(neig0[:,0]*meig0[:,0]))
for i in range(nrank):
    neig0[:,i]*=eigvJ0[i]
for j in range(1,N):
    neig0[:,j]*=eigvJ0[j]
## >>>>> eigendecomposition 
leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
''' The first '''
reig0=meig0.copy()
normval=np.sum(reig0*neig0.copy(),axis=0)
normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
leig0=neig0.copy()/normval.copy()
''' calculate s0'''
Z = np.zeros((NE*2-1,NE*2-1))
for i in range(NE*2-1):
    b1=eigvJ0[i+1]
    Z[i,i]=1/(b1-eigvJ0[0])
S0=reig0[:,1:]@Z@leig0[:,1:].T
print('test normal 2:',np.sum(leig0[:,0]*reig0[:,0]))
print('test normal 3:',np.sum(meig0[:,0]*neig0[:,0]))
print('eigenvalues:',eigvJ0[:nrank])
'''## for reference '''
x0,y0=np.reshape(reig0[:,0],(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
''' ## normalization 0 (reference) absolute reference is xAm, yAm '''
tildex,tildey=x0[:,0].copy(),y0[:,0].copy()
## make sure the E in x is positive, if negative then all x and y should change sign
if (np.mean(tildex[:NE])*xAm[0,0])>0:
    tildex*=(-1)
    tildey*=(-1)
x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
## >>>> renormalization, mu_l and mu_r are unchanged
x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey
x0,y0=np.reshape(x0,(NE*2,1)),np.reshape(y0,(NE*2,1))
'''
##>>>>>>>>>>> end >>>>>>>>
'''

'''
Network Setting for Iterating Following
'''
signeta=np.ones(3)
# signeta[1]*=(-1)
## heterogeneous random gain
''' >>>>>>>>>>>>>> Case 1 and Case 2 >>>>>>>>>>>>>>>>>>>>>>> '''
xee,xei,xie,xii=1.0,1.0,1.0,1.0 ## HOMOgeneous random gain g_kl
# xee,xei,xie,xii=1.0,0.5,0.2,0.8 ## HETEROgeneous random gain
# coeffeta=np.array([1.0,0.2,0.6]) ## HETERogeneous symmetry D_{\rho}
# coeffeta=np.array([1.0,0.8,1.0])
coeffeta=np.array([1.0,1.0,1.0])
# coeffeta=np.array([1.0,.0,.0])
# coeffeta = np.array([1.0,0.6,0.3])
# coeffeta=np.zeros(3)
# ppercent = np.array([0.8,0.6,0.4])
ppercent = np.ones(3)

'''
Recording Variables
'''
# original data
eigvalseries=np.zeros((ngavg,ntrial,N),dtype=complex)
Reigvecseries,Leigvecseries=np.zeros((ngavg,ntrial,NE*2),dtype=complex),np.zeros((ngavg,ntrial,NE*2),dtype=complex)
Beigvseries=np.zeros((ngavg,ntrial,N),dtype=complex)
''' All recorded connectivity statistics '''
statesReig,statesLeig=np.zeros((ngavg,ntrial,3,2),dtype=complex),np.zeros((ngavg,ntrial,3,2),dtype=complex) #E,I,Total/mean,std# 
coveigvecseries=np.zeros((ngavg,ntrial,3),dtype=complex) # E, I and total
''' All random matrices for each trials '''
## Not X total (symmetry), nut Y independent
gYbasexee=np.zeros((ntrial,NE*2,NE*2),dtype=complex)
gYbasexee=np.zeros((ntrial,NE*2,NE*2),dtype=complex)
SparseMat=np.zeros((ntrial,3,NE*2,NE*2))
''' Iterative Processing '''
etaseries=np.linspace(0.0,1.0,neta)
for iktrial in range(ntrial):
    '''
    ##>>>>>>>>>>> g0!=0, >>>>>>>>
    '''
    Xsym=np.random.randn(NE*2,NE*2)*gaverage/np.sqrt(2*NE) # /np.sqrt(2) no later
    Xsym[:NE,:NE]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[:NE,NE:]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[NE:,:NE]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[NE:,NE:]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    XsymT=Xsym.copy().T

    J0=Am.copy()+X0.copy()
    ''' # calculate the original \bar{m} and \bar{n} '''
    eigvJ0,eigrvecJ0=la.eig(J0)
    inveigrvecJ0=la.inv(eigrvecJ0)
    meig0 = np.squeeze(eigrvecJ0[:,:].copy())
    neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
    neig0=neig0.copy().T
    print('test normal:',np.sum(neig0[:,0]*meig0[:,0]))
    for i in range(nrank):
        neig0[:,i]*=eigvJ0[i]
    for j in range(1,N):
        neig0[:,j]*=eigvJ0[j]

    leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
    ''' The first '''
    reig0=meig0.copy()
    normval=np.sum(reig0*neig0.copy(),axis=0)
    normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
    leig0=neig0.copy()/normval.copy()
    ''' calculate s0'''
    Z = np.zeros((NE*2-1,NE*2-1))
    for i in range(NE*2-1):
        b1=eigvJ0[i+1]
        Z[i,i]=1/(b1-eigvJ0[0])
    S0=reig0[:,1:]@Z@leig0[:,1:].T
    print('test normal 2:',np.sum(leig0[:,0]*reig0[:,0]))
    print('test normal 3:',np.sum(meig0[:,0]*neig0[:,0]))
    print('eigenvalues:',eigvJ0[:nrank])
    '''## for reference '''
    x0,y0=np.reshape(reig0[:,0],(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
    ''' ## normalization 0 (reference) absolute reference is xAm, yAm '''
    tildex,tildey=np.reshape(x0[:,0].copy(),(NE*2,1)),np.reshape(y0[:,0].copy(),(NE*2,1))
    # make sure the E in x is positive, if negative then all x and y should change sign
    if (np.mean(tildex[:NE])*xAm[0,0])>0:
        tildex*=(-1)
        tildey*=(-1)
    x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
    x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
    y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey
    x0,y0=np.reshape(x0,(NE*2,1)),np.reshape(y0,(NE*2,1))

    ##>>>>>>>>>>>>>>>>> SPARSE INDICES >>>>>>>>>>>>>>>>>>>>>>
    ## EE CHOOSING
    hNE=int(ppercent[0]*NE)
    hrelist = np.random.permutation(NE)
    xh = np.squeeze(hrelist[:hNE])
    [xeehNE,yeehNE]=np.meshgrid(xh,xh)
    SparseMat[iktrial,0,xeehNE,yeehNE]=1

    ## EI IE CHOOSINGF
    hNE=int(ppercent[1]*NE)
    hrelist = np.random.permutation(NE)
    xh = np.squeeze(hrelist[:hNE])
    vrelist = np.random.permutation(NE)
    yh = np.squeeze(vrelist[:hNE])
    [xeihNE,yeihNE]=np.meshgrid(xh,yh)
    xeihNE +=NE
    SparseMat[iktrial,1,xeihNE,yeihNE]=1

    ## II CHOOSING
    hNE=int(ppercent[2]*NE)
    hrelist = np.random.permutation(NE)
    xh = np.squeeze(hrelist[:hNE])
    [xiihNE,yiihNE]=np.meshgrid(xh,xh)
    xiihNE,yiihNE = xiihNE+NE,yiihNE+NE
    SparseMat[iktrial,2,xiihNE,yiihNE]=1
    ## >>>>>>>>>>>>> IDENTICAL >>>>>>>>>

    for idxeta,eta in enumerate(etaseries):
        etaset=eta*np.ones(3)#np.zeros(3)#eta*np.ones(3)
        for icoeff in range(3):
          etaset[icoeff]*=coeffeta[icoeff]
        print('etaset:',ppercent*etaset)
        # etaset[0]*=0.1
        Xinit = Xsym.copy()
        
        ''' heterogeneous ETA method 1 '''
        '''
        '''
        ## EE ##
        asqr=(1-np.sqrt(1-etaset[0]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[:NE,:NE]=signeta[0]*aamp*XsymT[:NE,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,:NE].copy()
        ## EI IE##
        asqr=(1-np.sqrt(1-etaset[1]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,:NE]=signeta[1]*aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        ## II ##
        asqr=(1-np.sqrt(1-etaset[2]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,NE:]=signeta[2]*aamp*XsymT[NE:,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,NE:].copy()
        

        '''
        ### >>>>>>>>. half Exc sym >>>>>>>>>.
        hNE=int(ppercent[0]*NE)
        asqr=(1-np.sqrt(1-etaset[0]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[:hNE,:hNE]=signeta[0]*aamp*XsymT[:hNE,:hNE].copy()+np.sqrt(1-aamp**2)*Xsym[:hNE,:hNE].copy()
        ## EI IE##
        hNE=int(ppercent[1]*NE)
        asqr=(1-np.sqrt(1-etaset[1]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:NE+hNE,NE-hNE:NE]=signeta[1]*aamp*XsymT[NE:NE+hNE,NE-hNE:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:NE+hNE,NE-hNE:NE].copy()
        Xinit[NE-hNE:NE,NE:NE+hNE]=signeta[1]*aamp*XsymT[NE-hNE:NE,NE:NE+hNE].copy()+np.sqrt(1-aamp**2)*Xsym[NE-hNE:NE,NE:NE+hNE].copy()
        ## II ##
        hNE=int(ppercent[2]*NE)
        asqr=(1-np.sqrt(1-etaset[2]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:NE+hNE,NE:NE+hNE]=signeta[2]*aamp*XsymT[NE:NE+hNE,NE:NE+hNE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:NE+hNE,NE:NE+hNE].copy()
        '''

        
        # ### >>>>>>>>. half Exc sym >>>>>>>>>.
        # # # modified
        # # vrelist = np.random.permutation(NE)
        # # yh = np.squeeze(vrelist[:hNE])
        # # [xhNE,yhNE]=np.meshgrid(xh,yh)
        # asqr=(1-np.sqrt(1-etaset[0]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        # aamp=np.sqrt(asqr)
        # Xinit[xeehNE,yeehNE]=signeta[0]*aamp*XsymT[xeehNE,yeehNE].copy()+np.sqrt(1-aamp**2)*Xsym[xeehNE,yeehNE].copy()
        # ## EI IE##
        # asqr=(1-np.sqrt(1-etaset[1]**2))/2.0
        # aamp=np.sqrt(asqr)
        # Xinit[xeihNE,yeihNE]=signeta[0]*aamp*XsymT[xeihNE,yeihNE].copy()+np.sqrt(1-aamp**2)*Xsym[xeihNE,yeihNE].copy()
        # Xinit[yeihNE,xeihNE]=signeta[0]*aamp*XsymT[yeihNE,xeihNE].copy()+np.sqrt(1-aamp**2)*Xsym[yeihNE,xeihNE].copy()
        # ## II ##
        # asqr=(1-np.sqrt(1-etaset[2]**2))/2.0
        # aamp=np.sqrt(asqr)
        # Xinit[xiihNE,yiihNE]=signeta[0]*aamp*XsymT[xiihNE,yiihNE].copy()+np.sqrt(1-aamp**2)*Xsym[xiihNE,yiihNE].copy()
        

        ''' THEORETICAL FULL ETA '''
        symtheofull=gaverage**2*(etaset[0]*signeta[0]+etaset[2]*signeta[2]+2*etaset[1]*signeta[1])/4.0#+(a-b)**2/(NE*2)
        X=Xinit.copy()
        # incase heterogeneous
        X[:NE,:NE]*=(xee)
        X[:NE,NE:]*=(xei)
        X[NE:,:NE]*=(xie)
        X[NE:,NE:]*=(xii)

        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
        symnumfull*=2*NE
        print('compare degree of symmetry:',symnumfull,'--num; ',etaset*signeta,'--setting')
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:].copy())
        # meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
        neig=neig.copy().T
        print('test normal 4:',np.sum(neig[:,0]*meig[:,0]))
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
        for j in range(1,N):
            neig[:,j]*=eigvJ[j]
        ## >>> for low rank framework m and n
        eigvalseries[idxeta,iktrial,:]=eigvJ[:]
        
        ## >>> for eigendecomposition 
        Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
        ''' The first '''
        Reig=meig.copy()
        normval=np.sum(Reig*neig.copy(),axis=0)
        normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        Leig=neig.copy()/normval.copy()
        print('test normal 5:',np.sum(Leig[:,0]*Reig[:,0]))
        '''
        normalization 0 (reference)
        '''
        tildex,tildey=Reig[:,0].copy(),Leig[:,0].copy()
        if (np.mean(tildex[:NE])*xAm[0,0])>0:
            tildex*=(-1)
            tildey*=(-1)
        xnorm0=np.sqrt(np.squeeze(tildey.T@x0)/np.squeeze(y0.T@tildex))*tildex
        ynorm0=np.sqrt(np.squeeze(tildex.T@y0)/np.squeeze(x0.T@tildey))*tildey

        Beigvseries[idxeta,iktrial,:]=eigvJ.copy()
        Reigvecseries[idxeta,iktrial,:],Leigvecseries[idxeta,iktrial,:]=xnorm0[:].copy(),ynorm0[:].copy()

        '''  RECORDING STATES '''
        statesReig[idxeta,iktrial,0,0],statesReig[idxeta,iktrial,1,0]=np.mean(Reigvecseries[idxeta,iktrial,:NE]),np.mean(Reigvecseries[idxeta,iktrial,NE:])
        statesReig[idxeta,iktrial,2,0]=np.mean(Reigvecseries[idxeta,iktrial,:])
        statesReig[idxeta,iktrial,0,1],statesReig[idxeta,iktrial,1,1]=np.std(Reigvecseries[idxeta,iktrial,:NE]),np.std(Reigvecseries[idxeta,iktrial,NE:])
        statesReig[idxeta,iktrial,2,1]=np.std(Reigvecseries[idxeta,iktrial,:])
       
        statesLeig[idxeta,iktrial,0,0],statesLeig[idxeta,iktrial,1,0]=np.mean(Leigvecseries[idxeta,iktrial,:NE]),np.mean(Leigvecseries[idxeta,iktrial,NE:])
        statesLeig[idxeta,iktrial,2,0]=np.mean(Leigvecseries[idxeta,iktrial,:])
        statesLeig[idxeta,iktrial,0,1],statesLeig[idxeta,iktrial,1,1]=np.std(Leigvecseries[idxeta,iktrial,:NE]),np.std(Leigvecseries[idxeta,iktrial,NE:])
        statesLeig[idxeta,iktrial,2,1]=np.std(Leigvecseries[idxeta,iktrial,:])

        ''' covariance '''
        '''
        Note, we should change the normalization for the first item to the (Refer) normalization 
        tildeReig,tildeLeig
        normalization 0
        '''
        ## normalization 0 reference
        tildeReig,tildeLeig=np.squeeze(Reigvecseries[idxeta,iktrial,:]),np.squeeze(Leigvecseries[idxeta,iktrial,:])
        reig,leig=np.reshape(tildeReig[:].copy(),(NE*2,1)),np.reshape(tildeLeig[:].copy(),(NE*2,1))
        ## mean at eact point
        rEceneig,rIceneig = np.mean(reig[:NE]),np.mean(reig[NE:])
        lEceneig,lIceneig = np.mean(leig[:NE]),np.mean(leig[NE:])
        ## random components
        rrandeig,lrandeig=reig.copy(),leig.copy()
        rrandeig[:NE],rrandeig[NE:]=rrandeig[:NE]-rEceneig,rrandeig[NE:]-rIceneig
        lrandeig[:NE],lrandeig[NE:]=lrandeig[:NE]-lEceneig,lrandeig[NE:]-lIceneig
        coveigvecseries[idxeta,iktrial,0]=np.reshape(rrandeig[:NE],(NE,1)).T@np.reshape(lrandeig[:NE],(NE,1))
        coveigvecseries[idxeta,iktrial,1]=np.reshape(rrandeig[NE:],(NE,1)).T@np.reshape(lrandeig[NE:],(NE,1))
        coveigvecseries[idxeta,iktrial,2]=coveigvecseries[idxeta,iktrial,0]+coveigvecseries[idxeta,iktrial,1]
        ''' record the random derivative '''
        if idxeta == 0:
            print('recording!!!')
            ## log: there is a bug, X is the total random xconn, including symmetry
            gYbasexee[iktrial,:,:]=Xsym.copy()


* Recording PARAMETERS, VARIABLES and DATA

In [ ]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE
Randgain=np.zeros(4)
Randgain[0],Randgain[1],Randgain[2],Randgain[3]=xee,xei,xie,xii

np.savez(now+'_perturbationPureSubEEHetereta_data',x0=x0,y0=y0,xAm=xAm,yAm=yAm,Jparameters=Jparameters,Randgain=Randgain,gaverage=gaverage,etaseries=etaseries,etaset=etaset,signeta=signeta,eigvalseries=eigvalseries[:,:,0],Reigvecseries=Reigvecseries,Leigvecseries=Leigvecseries,Beigvseries=Beigvseries[:,:,0],meigvecAm=meigvecAm[:,0],neigvecAm=neigvecAm[:,0],eigvAm=eigvAm,statesReig=statesReig,statesLeig=statesLeig,coveigvecseries=coveigvecseries,gYbasexee=gYbasexee,SparseMat=SparseMat,coeffeta=coeffeta,ppercent=ppercent)


* supplemen -- order patterns

In [ ]:
'''
check the structure of XX and compare it with the theoretcial image
'''
Xfinal_random = X.copy()
kcut = 7
Xorder = np.zeros((kcut,NE*2,NE*2))
for i in range(kcut):
    Xorder[i,:,:] = la.matrix_power(Xfinal_random.copy(),i)
fig,ax=plt.subplots(2,3,figsize=(6,6))
BINS = 15
for i in range(2):
    for j in range(3):
        iorder = i*3+j
        ax[i][j].imshow(Xorder[iorder,NE-BINS:NE+BINS,NE-BINS:NE+BINS],cmap='gray',clim=([-.6,0.6]))

* EIGENSPECTRA (eigenbulk and eigenvalue outlier)

In [ ]:
'''
with symmetry, subcircuit
eigvJ0
'''
idxgavg,idxtrial=9,8 # 6,3,0
figtspt,axtspt=plt.subplots(figsize=(6,3))
cm='bgc'
idxc=0
axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,1:]),np.imag(Beigvseries[idxgavg,idxtrial,1:]),s=5,c=cm[idxc],alpha=0.25) # >>>>>>>>>>>>>>
axtspt.scatter(np.real(Beigvseries[idxgavg,:,0]),np.imag(Beigvseries[idxgavg,:,0])-0.2*idxc,s=20,c=cm[idxc],marker='^',alpha=0.5) # >>>>>>>>>>>>
axtspt.set_aspect('equal')
axtspt.plot(np.ones(2)*eigvAm[0],np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
axtspt.spines['right'].set_color('none')
axtspt.spines['top'].set_color('none')
axtspt.xaxis.set_ticks_position('bottom')
axtspt.spines['bottom'].set_position(('data', 0))

aee,aei,aie,aii=xee,xei,xie,xii
print(xee,xei,xie,xii)
idxtrial=10
eta=0
xticks = np.linspace(-1.3,1.3,2)
xlims = [-1.3,1.3]
yticks = np.linspace(-1.3,1.3,2)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
# first do not multiply at
ahomo=gaverage
xee_,xei_,xie_,xii_=ahomo*aee/np.sqrt(2),ahomo*aei/np.sqrt(2),ahomo*aie/np.sqrt(2),ahomo*aii/np.sqrt(2)
gmat = np.array([[xee_**2,xei_**2],[xie_**2,xii_**2]])
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
r_g = np.sqrt(r_g2)
print(r_g)
eta=etaseries[idxgavg]
longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
xr = longaxis*np.cos(theta)
yr = shortaxis*np.sin(theta)
axtspt.plot(xr, yr, color="gray", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>
axtspt.plot(-longaxis*np.ones(2), np.array([-1.0,1.0]), color="gray", linewidth=2.5,linestyle='--') # >>>>>
axtspt.set_title(r'$\eta=$'+np.str(eta))
xticks = np.linspace(-2.5,3.5,2)
xlims = [-2.5,3.5]
yticks = np.linspace(-1.0,1.0,2)
ylims = [-1.1,1.1]
axtspt.set_xlim(xlims)
axtspt.set_ylim(ylims)
axtspt.set_xticks(xticks)
axtspt.set_yticks(yticks)
axtspt.set_aspect('equal')

* eigenvalue outlier

In [ ]:
'''
determinant lemma, odd terms 
even order terms should be maitained
combination

!!! In sparse condition
'''
from scipy.special import comb, perm
gaverage
yticks = np.linspace(2.40,3.0,2)
ylims = [2.4,3.0]
xticks = np.linspace(-0.1,1.1,2)
xlims = [-0.1,1.1]

coeff=coeffeta.copy()
nAm=np.reshape(yAm*eigvAm[0],(NE*2,1))
mAm=np.reshape(xAm.copy(),(NE*2,1))
nAm,mAm=np.real(nAm),np.real(mAm)
y0E,y0I=np.mean(y0[:NE,0]),np.mean(y0[NE:,0])
x0E,x0I=np.mean(x0[:NE,0]),np.mean(x0[NE:,0])
lambda_theo2 = np.zeros(neta)
lambda_theo4 = np.zeros(neta)
lambda_num = np.mean(np.squeeze(Beigvseries[:,:,0]),axis=1)
cutoff = 2
nAmsparse,mAmsparse = nAm.copy(),mAm.copy()
for idxeta,eta in enumerate(etaseries):
    etaE,etaB,etaI=eta*coeff[0],eta*coeff[1],eta*coeff[2]
    '''
    Even-order terms have basic element XX.T(X.TX)
    '''
    bXXTEE,bXXTEI,bXXTIE,bXXTII=np.eye(NE),np.eye(NE),np.eye(NE),np.eye(NE)
    bXXTEE*=(etaE*signeta[0]*ppercent[0]+0)/4
    bXXTEI*=(0+etaB*signeta[1]*ppercent[1])/4
    bXXTIE*=(0+etaB*signeta[1]*ppercent[1])/4
    bXXTII*=(etaI*signeta[2]*ppercent[2]+0)/4
    # cut off = 2
    C=comb(2,1)
    print(C)
    moment2=np.zeros((NE*2,NE*2))
    moment2[:NE,:NE]=(bXXTEE*ppercent[0]+bXXTEI*ppercent[1]+0)
    moment2[NE:,NE:]=(0+bXXTIE*ppercent[1]+bXXTII*ppercent[2])
    print(np.shape(moment2))
    B2=np.squeeze(C*(nAm.T@(moment2)@mAm)*gaverage**2)
    coeff2 = [1,-eigvAm[0],0,-B2]
    print(coeff2)
    C=comb(4,2)
    ## EE sparse summation ppercent[0], EI, IE sparse summation ppercent[1]
    ## II sparse summation ppercent[2] II
    # >>>>>>> 4 order moment >>>>>>>>>>
    moment4 = np.zeros((NE*2,NE*2))
    moment4_add=np.zeros((NE*2,NE*2))
    moment4[:NE,:NE]=((bXXTEE*ppercent[0])**2*ppercent[0]+(bXXTEI*ppercent[1])**2*ppercent[1]+0)
    minnum_Etype = np.minimum(ppercent[0],ppercent[1])
    moment4_add[:NE,:NE]=minnum_Etype*(2*(bXXTEE*ppercent[0])*(bXXTEI*ppercent[1]))
    moment4[NE:,NE:]=(0+(bXXTIE*ppercent[1])**2*ppercent[1]+(bXXTII*ppercent[2])**2*ppercent[2])
    minnum_Itype = np.minimum(ppercent[2],ppercent[1])
    moment4_add[NE:,NE:]=minnum_Itype*(2*(bXXTII*ppercent[2])*(bXXTEI*ppercent[1]))
    moment4+=moment4_add
    B4=np.squeeze(C*(nAm.T@(moment4)@mAm)*gaverage**4)
    # B4=(nAm.T@(Y_t@YT_t@Y_t@YT_t)@mAm)*fory**2*foryt**2*6
    coeff4 = [1,-eigvAm[0],0,-B2,0,-B4]
    print(coeff4)
    lambda_test2=np.roots(coeff2)
    lambda_theo2[idxeta]=lambda_test2[0]
    lambda_test4=np.roots(coeff4)
    lambda_theo4[idxeta]=lambda_test4[0]
    print(lambda_test2[0],lambda_test4[0])
fig,ax=plt.subplots(figsize=(5,5))  
ax.plot(lambda_num,lambda_theo2,'b',label='2 cut off')
ax.plot(lambda_num,lambda_theo4,'r',label='4 cut off')
# ax.plot(lambda_num,lambda_theo,'b')
ax.plot(lambda_num,lambda_num,'k',label='real')
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')
ax.legend()

fig,ax=plt.subplots(figsize=(5,3))  
ax.plot(etaseries,lambda_theo2,color='b',linewidth=1.5,label='theo')
ax.plot(etaseries,lambda_num,color='b',linewidth=1.5,linestyle='--',label='num')
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
# ax.set_aspect('equal')
ax.legend()
# # ax.scatter(np.real(Beigvseries[neta-5,8,1:]),np.imag(Beigvseries[neta-5,8,1:]))
# # ax.set_aspect('equal')


In [ ]:
'''
loading patterns
for ensemble-averaging results
>>>>>>>>>>>>>>>>> with eigenvalues >>>>>>>>>>>>>>>
'''
# xticks = np.linspace(-0.2,0.25,2)
# xlims = [-0.2,0.25]
# yticks = np.linspace(-0.2,0.25,2)
# ylims = [-0.2,0.25]
xticks = np.linspace(-0.15,0.10,2)
xlims = [-0.15,0.10]
yticks = np.linspace(-0.06,0.01,2)
ylims = [-0.06,0.01]
theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''

idxtrial=5#16#
idxeta=9#9# 
alphaval=0.10
edgv='black'
cm='br'
fig,ax=plt.subplots(figsize=(5,3))  
nrank=1
ntrial = np.shape(Reigvecseries)[1]
NE=int(NE)
print(NE)
'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)

mEvec,mIvec,nEvec,nIvec=np.squeeze(Reigvecseries[idxeta,:,:NE]),np.squeeze(Reigvecseries[idxeta,:,NE:]),np.squeeze(Leigvecseries[idxeta,:,:NE]),np.squeeze(Leigvecseries[idxeta,:,NE:])
mEvec,mIvec,nEvec,nIvec=mEvec.flatten(),mIvec.flatten(),nEvec.flatten(),nIvec.flatten()
scale_std=3.0
for irank in range(nrank):
    meanmEtotal,stdmEtotal = np.mean(mEvec),np.std(mEvec)
    varmE = mEvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEvec[idxwhere]=meanmEtotal
    meanmE[irank]=np.mean(mEvec)

    # puring I
    meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
    varmI = mIvec - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mIvec[idxwhere]=meanmItotal
    meanmI[irank]=np.mean(mIvec)
    
    # n vector
    # puring E
    meannEtotal,stdnEtotal = np.mean(nEvec),np.std(nEvec)
    varnE = nEvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEvec[idxwhere]=meannEtotal
    meannE[irank]=np.mean(nEvec)

    # puring I
    meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
    varnI = nIvec - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nIvec[idxwhere]=meannItotal
    meannI[irank]=np.mean(nIvec)

''' calculate directions '''
noiseE,noiseI = np.zeros((NE*ntrial,2,nrank)),np.zeros((NE*ntrial,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI)  

ax.scatter(nIvec,mIvec,s=5.0,c='b',alpha=alphaval)#cm[1],alpha=alphaval)
ax.scatter(nEvec,mEvec,s=5.0,c='r',alpha=alphaval)#cm[0],alpha=alphaval)

# ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color='gray',linestyle='--',linewidth=1.5)
# ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color='gray',linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color=edgv,linestyle='--',linewidth=1.5)

confidence_ellipse(np.real(nEvec),np.real(mEvec),ax,edgecolor=edgv)
confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor=edgv)
ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')
print(etaseries[idxeta])

* overlap/covariance between eigenvectors' perturbations

In [ ]:
'''
in the case with reciprical connectivity,
with certain degree of homogeneous symmetry

we integrating from 0 to eta and get the CHANGING 
'''
DeltaL_theo,DeltaR_theo,DeltaL_num,DeltaR_num=np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex) 
DeltaLR_math,DeltaLR_theo,DeltaLR_num = np.zeros((neta,3),dtype=complex),np.zeros((neta,ntrial),dtype=complex),np.zeros((neta,ntrial,3),dtype=complex)

Jinit = np.zeros((ntrial,NE*2,NE*2),dtype=complex)
## Jinit same for identical trial # realization
Leigvecinit,Reigvecinit = np.zeros((ntrial,NE*2),dtype=complex),np.zeros((ntrial,NE*2),dtype=complex)
## initial vectors LEFT & RIGHT same for identical trial

for itrial in range(ntrial):
    # initial eigenvectors
    Leigvecinit[itrial,:],Reigvecinit[itrial,:]=np.squeeze(Leigvecseries[0,itrial,:]).copy(),np.squeeze(Reigvecseries[0,itrial,:]).copy()
    l0,r0=np.reshape(Leigvecinit[itrial,:],(2*NE,1)),np.reshape(Reigvecinit[itrial,:],(2*NE,1))

    for idxeta in range(neta):
        ## choose one specific degree of symmetry
        etaloop = etaseries[idxeta]*np.ones(3)
        # print('eta:',etaloop)
        etaloop=etaloop*coeffeta
        # etaloop=(etaloop*(ppercent**2)
        print('equivalent etaloop:',etaloop)

        asqr=(1-np.sqrt(1-etaloop[0]**2))/2.0 
        rhoE=asqr

        asqr=(1-np.sqrt(1-etaloop[1]**2))/2.0 
        rhoB=asqr

        asqr=(1-np.sqrt(1-etaloop[2]**2))/2.0 
        rhoI=asqr

        # numerical value
        changeL_num = np.reshape(np.squeeze(Leigvecseries[idxeta,itrial,:]),(NE*2,1))-l0#np.mean(np.squeeze(Leigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Leigvecseries[0,itrial,:]),(NE*2,1))
        changeR_num = np.reshape(np.squeeze(Reigvecseries[idxeta,itrial,:]),(NE*2,1))-r0#np.mean(np.squeeze(Reigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Reigvecseries[0,itrial,:]),(NE*2,1))
        DeltaL_num[idxeta,itrial,:],DeltaR_num[idxeta,itrial,:]=np.squeeze(changeL_num),np.squeeze(changeR_num)
        DeltaLR_num[idxeta,itrial,2]=np.squeeze(changeL_num.T@changeR_num)
        DeltaLR_num[idxeta,itrial,0]=np.sum(changeL_num[:NE,0]*changeR_num[:NE,0])
        DeltaLR_num[idxeta,itrial,1]=np.sum(changeL_num[NE:,0]*changeR_num[NE:,0])

        PIrho,PIrhominusrho=np.zeros((NE*2,NE*2)),np.zeros((NE*2,NE*2))
        PIrhoE,PIrhominusrhoE=np.zeros((NE*2,NE*2)),np.zeros((NE*2,NE*2))
        PIrhoI,PIrhominusrhoI=np.zeros((NE*2,NE*2)),np.zeros((NE*2,NE*2))
        hNE = int(NE*ppercent[0]) # E!
        PIrho[:hNE,:hNE]+=np.eye(hNE)*(NE*ppercent[0])*(np.sqrt(rhoE)+0)/NE/2.0
        PIrhominusrho[:hNE,:hNE]+=np.eye(hNE)*(NE*ppercent[0])*(np.sqrt((1-rhoE)*rhoE)+0)/NE/2.0

        PIrhoE[:hNE,:hNE]+=np.eye(hNE)*(NE*ppercent[0])*(np.sqrt(rhoE)+0)/NE/2.0
        PIrhominusrhoE[:hNE,:hNE]+=np.eye(hNE)*(NE*ppercent[0])*(np.sqrt((1-rhoE)*rhoE)+0)/NE/2.0
        hNE = int(NE*ppercent[1]) # I!
        PIrho[:hNE,:hNE]+=np.eye(hNE)*(NE*ppercent[1])*(0+np.sqrt(rhoB))/NE/2.0
        PIrhominusrho[:hNE,:hNE]+=np.eye(hNE)*(NE*ppercent[1])*(0+np.sqrt((1-rhoB)*rhoB))/NE/2.0

        PIrhoI[:hNE,:hNE]+=np.eye(hNE)*(NE*ppercent[1])*(0+np.sqrt(rhoB))/NE/2.0
        PIrhominusrhoI[:hNE,:hNE]+=np.eye(hNE)*(NE*ppercent[1])*(0+np.sqrt((1-rhoB)*rhoB))/NE/2.0
        
        hNE = int(NE*ppercent[1]) # E!
        PIrho[NE:NE+hNE,NE:NE+hNE]+=np.eye(hNE)*(NE*ppercent[1])*(np.sqrt(rhoB)+0)/NE/2.0
        PIrhominusrho[NE:NE+hNE,NE:NE+hNE]+=np.eye(hNE)*(NE*ppercent[1])*(np.sqrt((1-rhoB)*rhoB)+0)/NE/2.0
        PIrhoE[NE:NE+hNE,NE:NE+hNE]+=np.eye(hNE)*(NE*ppercent[1])*(np.sqrt(rhoB)+0)/NE/2.0
        PIrhominusrhoE[NE:NE+hNE,NE:NE+hNE]+=np.eye(hNE)*(NE*ppercent[1])*(np.sqrt((1-rhoB)*rhoB)+0)/NE/2.0
        hNE = int(NE*ppercent[2]) # I!
        PIrho[NE:NE+hNE,NE:NE+hNE]+=np.eye(hNE)*(NE*ppercent[2])*(0+np.sqrt(rhoI))/NE/2.0
        PIrhominusrho[NE:NE+hNE,NE:NE+hNE]+=np.eye(hNE)*(NE*ppercent[2])*(np.sqrt((1-rhoI)*rhoI)+0)/NE/2.0

        PIrhoI[NE:NE+hNE,NE:NE+hNE]+=np.eye(hNE)*(NE*ppercent[2])*(0+np.sqrt(rhoI))/NE/2.0
        PIrhominusrhoI[NE:NE+hNE,NE:NE+hNE]+=np.eye(hNE)*(NE*ppercent[2])*(np.sqrt((1-rhoI)*rhoI)+0)/NE/2.0  

        DeltaLR_math[idxeta,2]=np.reshape(l0,(1,NE*2))@(2*PIrhominusrho-2*PIrho)@np.reshape(r0,(2*NE,1))*gaverage**2/eigvAm[0]**2
        # Excitatory
        DeltaLR_math[idxeta,0]=np.reshape(l0,(1,NE*2))@(2*PIrhominusrhoE-2*PIrhoE)@np.reshape(r0,(2*NE,1))*gaverage**2/eigvAm[0]**2

        DeltaLR_math[idxeta,1]=np.reshape(l0,(1,NE*2))@(2*PIrhominusrhoI-2*PIrhoI)@np.reshape(r0,(2*NE,1))*gaverage**2/eigvAm[0]**2 #(2*aampT*aampO*gaverage**2-2*aampT*gaverage**2)/eigvAm[0]**2
        # if idxeta==neta-1:
        #     DeltaLR_math[idxeta,0]=-DeltaLR_math[idxeta,0]
        #     DeltaLR_math[idxeta,1]=-DeltaLR_math[idxeta,1]
        # DeltaLR_math[idxeta,2]=DeltaLR_math[idxeta,0]+DeltaLR_math[idxeta,1]

In [ ]:
'''
Total Theoretical Solution for Covariance
'''

'''
in the case with reciprical connectivity,
with certain degree of homogeneous symmetry

we integrating from 0 to eta and get the CHANGING 
'''
DeltaL_theo,DeltaR_theo,DeltaL_num,DeltaR_num=np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex) 
DeltaLR_math,DeltaLR_theo,DeltaLR_num = np.zeros((neta,3),dtype=complex),np.zeros((neta,ntrial),dtype=complex),np.zeros((neta,ntrial,3),dtype=complex)

Jinit = np.zeros((ntrial,NE*2,NE*2),dtype=complex)
## Jinit same for identical trial # realization
Leigvecinit,Reigvecinit = np.zeros((ntrial,NE*2),dtype=complex),np.zeros((ntrial,NE*2),dtype=complex)
## initial vectors LEFT & RIGHT same for identical trial

for itrial in range(ntrial):
    # initial eigenvectors
    Leigvecinit[itrial,:],Reigvecinit[itrial,:]=np.squeeze(Leigvecseries[0,itrial,:]).copy(),np.squeeze(Reigvecseries[0,itrial,:]).copy()
    l0,r0=np.reshape(Leigvecinit[itrial,:],(2*NE,1)),np.reshape(Reigvecinit[itrial,:],(2*NE,1))#np.reshape(-yAm,(2*NE,1)),np.reshape(-xAm,(2*NE,1))#

    for idxeta in range(neta):
        ## choose one specific degree of symmetry
        etaloop = etaseries[idxeta]*np.ones(3)
        # print('eta:',etaloop)
        etaloop=etaloop*coeffeta
        # etaloop=(etaloop*(ppercent**2)
        print('equivalent etaloop:',etaloop)

        asqr=(1-np.sqrt(1-etaloop[0]**2))/2.0 
        rhoE=asqr

        asqr=(1-np.sqrt(1-etaloop[1]**2))/2.0 
        rhoB=asqr

        asqr=(1-np.sqrt(1-etaloop[2]**2))/2.0 
        rhoI=asqr

        # numerical value
        changeL_num = np.reshape(np.squeeze(Leigvecseries[idxeta,itrial,:]),(NE*2,1))-l0#np.mean(np.squeeze(Leigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Leigvecseries[0,itrial,:]),(NE*2,1))
        changeR_num = np.reshape(np.squeeze(Reigvecseries[idxeta,itrial,:]),(NE*2,1))-r0#np.mean(np.squeeze(Reigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Reigvecseries[0,itrial,:]),(NE*2,1))
        DeltaL_num[idxeta,itrial,:],DeltaR_num[idxeta,itrial,:]=np.squeeze(changeL_num),np.squeeze(changeR_num)
        DeltaLR_num[idxeta,itrial,2]=np.squeeze(changeL_num.T@changeR_num)
        DeltaLR_num[idxeta,itrial,0]=np.sum(changeL_num[:NE,0]*changeR_num[:NE,0])
        DeltaLR_num[idxeta,itrial,1]=np.sum(changeL_num[NE:,0]*changeR_num[NE:,0])
   
        # EXCITATORY
        # JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[0]**2)*(np.sqrt((1-rhoE)*rhoE)-np.sqrt(rhoE)    
        # -JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[1]**2)*(np.sqrt((1-rhoB)*rhoB)-np.sqrt(rhoB)    
        DeltaLR_math[idxeta,0]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[0]**2)*(np.sqrt((1-rhoE)*rhoE)-np.sqrt(rhoE))    
        DeltaLR_math[idxeta,0]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[1]**2)*(np.sqrt((1-rhoB)*rhoB)-np.sqrt(rhoB)))
        # Inhibitory
        DeltaLR_math[idxeta,1]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[1]**2)*(np.sqrt((1-rhoB)*rhoB)-np.sqrt(rhoB))    
        DeltaLR_math[idxeta,1]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[2]**2)*(np.sqrt((1-rhoI)*rhoI)-np.sqrt(rhoI)))

        DeltaLR_math[idxeta,2]=DeltaLR_math[idxeta,1]+DeltaLR_math[idxeta,0]

In [ ]:
fig,ax=plt.subplots(figsize=(5,5))
cl='rbk'
for i in range(3):
    ax.plot(DeltaLR_math[:,i],np.mean(np.squeeze(DeltaLR_num[:,:,i]),axis=1),color=cl[i],linewidth=1.5,label='num v.s. theo')
# ax.plot(np.mean(DeltaLR_theo,axis=1),np.mean(DeltaLR_num,axis=1),color='b',linewidth=1.5,label='num v.s. simp')
ax.set_aspect('equal')

fig,ax=plt.subplots(figsize=(5,5))
cl='rbk'
for i in range(3):
    ax.plot(etaseries,DeltaLR_math[:,i],color=cl[i],linewidth=1.5)
    ax.plot(etaseries,np.mean(np.squeeze(DeltaLR_num[:,:,i]),axis=1),color=cl[i],linewidth=1.5,linestyle='--')
fig,ax=plt.subplots(figsize=(5,5))
cl='rbk'
for i in range(2,3):
    ax.plot(etaseries,DeltaLR_math[:,i],color=cl[i],linewidth=1.5)
    ax.plot(etaseries,np.mean(np.squeeze(DeltaLR_num[:,:,i]),axis=1),color=cl[i],linewidth=1.5,linestyle='--')
